# The maximal subgroups of the Monster

Python code accompanying the above paper by Heiko Dietrich, Melissa Lee, and Tomasz Popiel

https://arxiv.org/abs/2304.14646

Date: 6 December 2023

Warnings: 
* Some cells take a long time to run! (Expected timings are given in some cases; these were estimated using a 2022 "M2" Macbook Air.)
* Some variable names are repeated for consistency with the notation in the paper, so this file should be read in order/section-by-section.

## Preliminary code

Import required packages, including mmgroup: https://github.com/Martin-Seysen/mmgroup

In [1]:
from mmgroup import *
import timeit
import sys
import itertools as it
from functools import reduce
import numpy as np
import time
import random
import math
from itertools import islice
import multiprocessing as mp

A function to calculate the group commutator $[a,b] = a^{-1}b^{-1}ab$.

In [2]:
def Comm(a,b):
   return a**(-1)*b**(-1)*a*b

## Code accompanying Section 2

### Sections 2.4 and 2.5

The standard generators for $\mathbf{G} \cong 2^{1+24}.\mathrm{Co}_1$ defined in Section 2.5.

In [3]:
a = MM("M<y_2feh*x_51h*d_6f2h*p_199553794*l_2*p_1900800*l_2*p_684120>")
b = MM("M<y_32bh*x_0e4h*d_30fh*p_81928987*l_2*p_2880*l_1*p_21312*l_1*p_10455360>")

A function for verifying that the above elements are indeed standard generators for $\mathbf{G}$ (by checking the conditions described in Section 2.5).

In [4]:
def verify_std_gens_G(a,b):
   # the central element in mmgroup's copy of G = 2^{1+24}.Co_1
   z = MM("M<x_1000h>")
   if not (Comm(a**2,b**3) == z and a*z == z*a and b*z == z*b):
       return False
   # check membership in G_x0
   c0 = a.in_G_x0() and b.in_G_x0()
   # check orders
   c1 = a.order()==4 and b.order()==6
   # check G_x0-classes
   c2 = a.chi_G_x0()[1]==-13 and b.chi_G_x0()[1]==2 and (b.chi_G_x0()[0]==-3 or b.chi_G_x0()[0]==5)
   # check product in right class
   c3 = (a*b).order()==40 and ((a*b)**-1).chi_G_x0()[1]==0 and ((a*b)**-1).chi_G_x0()[2]*((a*b)**-1).chi_G_x0()[3]==0
   # check that element powers are not central in the subgroup generated by a and b
   c4 = (a**2)*b != b*(a**2) and (b**2)*a != a*(b**2) and (b**3)*a != a*(b**3)
   # check that a*b*a*(b^2) has order 6 in the quotient
   # choices are 6 or 21, so if the latter, returns False
   c5 = (a*b*a*b**2).order() % 6 ==0
   if not (c0 and c1 and c2 and c3 and c4 and c5):
       return False
   # check that we can generate Q: construct j0,..,j23
   myels = [(a**2)**((a*b)**i) for i in range(0,24)]
   # elements need to lie in Q, commute modulo the centre, and have order 2
   for i in range(0,24):
       if not myels[i].in_Q_x0():
           return False
       for j in range(i+1,24):
           if (myels[i]*myels[j] != myels[j]*myels[i]) and Comm(myels[i],myels[j]) != z:
               return False
           if myels[i].order() != 2:
               return False
   print("all good so far; now check size of Q;")
   print("this will take a long time")
   # check that ((myels[0]**i0)* ... *(myels[23]**i23)).order() == 1 mod <z> only for i0==...==i23=0
   s=0
   tups = it.product([0,1], repeat=24)
   for t in tups:
       if np.sum(t)==0:
           continue
       if s % 10000 == 0:
           print("Done ", s, " of ",2**24," tuples: ",100*s/2**24,"%", end='\r')
       terms = [myels[i]**t[i] for i in range(24)]
       prod = reduce((lambda x, y: x * y), terms)
       if prod.order() ==1 or (prod*z).order()==1:
           print("Found identity with tuple: ", t)
           return False
       s+=1
   print("all tests ok")
   return True

Check that verify_std_gens_G(a,b) returns True. 

***Warning: this takes some time and is not needed for any subsequent calculations, so has been commented out.***

In [5]:
# verify_std_gens_G(a,b)

An implementation of the homomorphism $\pi : \mathbf{G} \to \mathrm{GL}_2(24)$ with $\pi(\mathbf{G}) \cong \mathrm{Co}_1$ defined in Section 2.4.

In [6]:
def elt_to_24_mat(g):
   mat = []
   for i in range(24):
       x = generators.gen_leech2_op_word_leech2(2**(23-i),g.mmdata,len(g.mmdata),0)
       mat.append([int(d) for d in format((x %2**24), '#026b')[2:]])
   return mat

Magma code for confirming that the matrices $A$ and $B$ computed as elt_to_24_mat(a) and elt_to_24_mat(b) generate $\mathrm{Co}_1 < \mathrm{GL}_2(24)$ can be found at:

https://github.com/melissa-maths/M/blob/main/Co1.m

## Section 2.6

An implementation of the product replacement algorithm to produce 'random' elements of a subgroup of $\mathbf{M}$ from a generating set. 
* The input L is a list of mmgroup elements; the function outputs an element of the subgroup of $\mathbf{M}$ generated by L.
* The input n is the number of product replacement steps that are performed before returning an element.
* The input init_already is an optional argument. If set to False, then 200 product replacements are performed and a random element of the resulting list is returned, regardless of the choice of n.

***Warning: this function alters the input list L in-place!***

In [7]:
def get_random(L, n=1, init_already=True):
    i = 0
    if init_already == False:
        for i in range(6):
            L.append(random.choice(L))    
        return get_random(L,200)
    while i<n:
        a = random.choice(range(0,len(L)))
        b = a
        while b == a:
            b = random.choice(range(0,len(L)))
        if random.choice([0,1]):
            L[a] = L[a]*L[b]
        else:
            L[a] = L[b]*L[a]
        i = i+1
    return L[a]

A function that constructs a subgroup of $\mathbf{M}$ from a generating set.
* The input L is a list of mmgroup elements; the function returns a list of the elements comprising the subgroup of $\mathbf{M}$ generated by L.
* The input n is a 'desired' upper bound on the order of the subgroup generated by L; if more elements are found, then the function aborts.
* If the optional argument order_only is set to True, then the function only returns the order of the subgroup generated by L (and not the elements comprising this subgroup).

In [8]:
def group_generated_by(L, n, order_only=False):
    start = time.time()
    orb = [L[0]]
    orbset = {tuple(L[0].as_tuples())}
    os = 0
    for el in L:
        eltup = tuple(el.as_tuples())
        if not eltup in orbset:
            orb.append(el)
            orbset.add(eltup)
            os = os+1;
             
    j = 0
    while j <= os:
        for g in L:
            el= orb[j]*g
            eltup = tuple(el.as_tuples())
            if not eltup in orbset:
                orb.append(el)
                orbset.add(eltup)
                os = os+1;
                    
        j = j+1
        end = time.time()       
        print("Limit", n, "; have", os, "in time ", round(end-start,4), end='\r')
        if len(orbset)>n:
            print("Group is larger than imposed limit -- abort in time ", round(end-start,4) )
            return False
    
    print("Limit", n, "; have", len(orb), "in time ", round(end-start,4), end='\r')
    if order_only:
        return len(orb)
    return orb

## Code accompanying Section 3

### Proof of Proposition 3.1

The elements from Listing 5, and a check that $g_{13}$, $y_6$, $x_6$, and $x_3$ have orders $13$, $6$, $6$, and $3$.

In [9]:
g13 = MM("M<y_519h*x_0cb8h*d_3abh*p_178084032*l_2*p_2344320*l_2*p_471482*l_1*t_1*l_2*p_2830080*l_2*p_22371347*l_2*t_2*l_1*p_1499520*l_2*p_22779365*l_2*t_1*l_2*p_2597760*l_1*p_11179396*t_1*l_1*p_1499520*l_2*p_85838017*t_2*l_1*p_1499520*l_1*p_64024721*t_2*l_2*p_2386560*l_2*p_21335269>")
y6 = MM("M<y_4fh*x_1331h*d_0d46h*p_79853974*l_2*p_1943040*l_2*p_2398522*t_1*l_2*p_2344320*l_2*p_1858757*l_2*t_1*l_1*p_960*l_2*p_3120*l_2*p_517440*t_2*l_2*p_2597760*l_1*p_12132032*t_2*l_2*p_2880*l_1*p_465840*l_1*p_1565760*t_1*l_2*p_960*l_1*p_63994992*t_1*l_1*p_2027520*l_1*p_50146>")
c = MM("M<y_0fh*x_0bc4h*d_59h*p_207376512*l_2*p_1943040*l_2*p_22272232*l_2*t_1*l_1*p_1499520*l_1*p_22439*l_1*t_1*l_1*p_1394880*l_1*p_21456*l_2*p_4776960*t_2*l_1*p_1499520*l_2*p_53357227*t_1*l_2*p_960*l_2*p_10665792*l_1*p_6086400*t_1*l_2*p_1943040*l_2*p_64043939*t_2*l_2*p_2956800*l_1*p_64017049>")
d = MM("M<y_5a8h*x_0bcdh*d_941h*p_205645390*l_2*p_2830080*l_2*p_8690*l_2*t_1*l_2*p_1900800*l_2*p_10675420*t_1*l_2*p_2597760*l_1*p_42728016*t_2*l_2*p_2597760*l_1*p_10729207*t_1*l_2*p_1985280*l_1*p_21338086*t_2*l_2*p_2597760*l_1*p_21359269*t_1*l_1*p_1499520*l_1*p_42755907>")
x6 = MM("M<y_44eh*x_1906h*d_2d9h*p_173881751*l_1*p_2640000*l_1*p_1925314*l_1*t_1*l_1*p_2999040*l_1*p_2392772*l_1*t_1*l_1*p_1499520*l_1*p_32461673*l_1*t_1*l_2*p_2344320*l_2*p_84794*t_2*l_2*p_2956800*l_1*p_85413707*t_2*l_2*p_1985280*l_1*p_96477721*t_1*l_2*p_1985280*l_1*p_64023741>")
x3 = MM("M<y_492h*x_1fdeh*d_57ch*p_149571126*l_2*p_2787840*l_2*p_12996808*l_2*t_2*l_2*p_1943040*l_2*p_1463363*l_1*t_1*l_2*p_1394880*l_2*p_21312*l_1*p_1524480*t_2*l_2*p_2386560*l_2*p_21468896*t_2*l_1*p_1499520*l_1*p_956371*l_1*t_2*l_1*p_1920*l_2*p_467856*l_2*p_10315200>")

g13.order(), y6.order(), x6.order(), x3.order()

(13, 6, 6, 3)

A function that checks whether the cyclic group generated by $y_6$ acts faithfully on the cyclic group generated by $g_{13}$.

***This function is also used below for other cases.***

In [10]:
def is_faithful(g,y):
    orbit = []
    for i in range(y.order()):
        if g**(y**i) not in orbit:
            orbit.append(g**(y**i))
    return len(orbit) == y.order()

is_faithful(g13,y6)

True

Check that $g_{13}$ and $y_6$ commute with $c$ and $d$.

***Note that MM("M<1>") is the identity element of $\mathbf{M}$ in mmgroup.***

In [11]:
Comm(g13,c) == Comm(g13,d) == Comm(y6,c) == Comm(y6,d) == MM("M<1>")

True

Construct the subgroup $H = \langle c,d \rangle$ of $\mathbf{M}$, and check that it has order $5616 = \mathrm{PSL}_3(3)$.

***This takes 2-3 minutes.***

In [12]:
H = group_generated_by([c,d], 10000)

In [13]:
len(H)

5616

To speed up certain calculations, also create a version of H in "tuples" format.

In [14]:
H_tuples = [h.as_tuples() for h in H]

Confirm that $g_{13} \not \in H$, so that $H \cong \text{PSL}_3(3)$.

In [15]:
g13.as_tuples() in H_tuples

False

### Proof of Proposition 3.2

Check that $g_6 = y_gx$ has order $6$ and acts faithfully on $g_{13}$ for each $x \in X := \{1,x_6,x_6^2,x_6^3,x_3\}$.

In [16]:
X = [1,x6,x6**2,x6**3,x3]
all([(y6*x).order() == 6 and is_faithful(g13,y6*x) for x in X])

True

Check that $x_6$ and $x_3$ lie in $H$.

***Here we use the "tuples" version of $H$ for efficiency; a direct check is also possible (commented out) but takes longer.***

In [17]:
# Direct check (takes longer - a minute or so)
# x6 in H and x3 in H

x6.as_tuples() in H_tuples and x3.as_tuples() in H_tuples

True

Check that each $g_6 = y_6x$ cubes to a $2\text{B}$-involution.

In [18]:
all([((y6*x)**3).conjugate_involution()[0] == 2 for x in X])

True

Conjugate each $g_6$ into $\mathrm{G}$ (and check containment) and calculate its $\chi_\mathrm{M}$-value.

In [19]:
for x in X:
    h_x = ((y6*x)**3).conjugate_involution()[1]
    print(((y6*x)**h_x).in_G_x0(), ((y6*x)**h_x).chi_G_x0()[0])
    
# Delete the variable h_x
del h_x

True 77
True 5
True 5
True 5
True -1


### Code related to Remark 3.4

Here we list our generators for the subgroups "$Y$" of the normalisers of $\langle g_6 \rangle$ for $g_6 = y_6x$. The involutions "$j_2$" inverting the $g_6$ can be found via random search in these subgroups using e.g. the function get_random defined above. (We do not provide these involutions here because the files in which we have stored them are up to 8GB in size.)

The following function checks whether $h \in \mathrm{M}$ normalises (the cyclic subgroup generated) by $g \in \mathrm{M}$.

In [20]:
def is_normalised(g,h):
    cyclic_g = [g**i for i in range(g.order())]
    return all([x**h in cyclic_g for x in cyclic_g])

Elements normalising $g_6 = y_6$ (and check).

In [21]:
y6_normaliser =[MM("M<y_5ddh*x_0cb8h*d_780h*p_101185443*l_1*p_1499520*l_1*p_6864*t_1*l_2*p_2956800*l_1*p_21865340*l_1*t_1*l_2*p_1858560*l_1*p_2112*l_2*p_975360*t_1*l_1*p_2417280*t_1*l_2*p_2830080*l_2*p_43634325*t_2*l_1*p_1499520*l_1*p_42734752*t_2*l_2*p_1985280*l_1*p_42677955>"),
MM("M<y_0ch*x_17dh*d_0a73h*p_192340543*l_2*p_2597760*l_1*p_43686374*t_2*l_2*p_2344320*l_2*p_467749*l_2*t_2*l_1*p_2640000*l_1*p_13037268*l_1*t_2*l_2*p_2344320*l_2*p_13458*t_2*l_2*p_2830080*l_2*p_64046787*t_2*l_1*p_1499520*l_2*p_63994958*t_1*l_2*p_1900800*l_2*p_139586*t_1*l_2*p_2386560*l_2*p_42727145>"),
MM("M<y_4fh*x_1331h*d_0d46h*p_79853974*l_2*p_1943040*l_2*p_2398522*t_1*l_2*p_2344320*l_2*p_1858757*l_2*t_1*l_1*p_960*l_2*p_3120*l_2*p_517440*t_2*l_2*p_2597760*l_1*p_12132032*t_2*l_2*p_2880*l_1*p_465840*l_1*p_1565760*t_1*l_2*p_960*l_1*p_63994992*t_1*l_1*p_2027520*l_1*p_50146>"),
MM("M<y_8dh*x_12aah*d_0e02h*p_64563918*l_2*p_2830080*l_2*p_32088530*t_2*l_2*p_2344320*l_2*p_12149352*l_1*t_2*l_1*p_1415040*l_1*p_10667856*l_2*p_4796160*t_2*l_1*p_1499520*l_2*p_53357163*t_1*l_2*p_960*l_2*p_464928*l_2*p_549120*t_1*l_1*p_1105920*l_2*t_2*l_2*p_1943040*l_2*p_85812172>")]

all([is_normalised(y6, h) for h in y6_normaliser])

True

Elements normalising $g_6 = y_6x_6$.

In [22]:
y6x6_normaliser = [MM("M<y_6bah*x_40fh*d_8f6h*p_71067893*l_1*p_1499520*l_2*p_33443096*l_1*t_2*l_1*p_2027520*l_1*p_2859317*t_1*l_2*p_2597760*l_1*p_43159057*t_2*l_2*p_2597760*l_1*p_170651330*t_1*l_2*p_2830080*l_2*p_11602755*l_1*t_1*l_2*p_2386560*l_2*p_85371397*t_1*l_1*p_1499520*l_1*p_42756875>"),
MM("M<y_4c5h*x_194ah*d_5cfh*p_91427851*l_2*p_49272960*l_2*p_212488368*t_2*l_1*p_2640000*l_1*p_11598883*l_1*t_2*l_1*p_1457280*l_2*p_32476051*l_2*t_2*l_1*p_2027520*l_1*p_521506*t_1*l_2*p_2956800*l_1*p_106663200*t_1*l_2*p_2597760*l_1*p_42729988*t_1*l_1*p_1651200>"),
MM("M<y_92h*x_0dc5h*d_73fh*p_149085778*l_2*p_79875840*l_2*p_87859296*t_1*l_2*p_49716480*l_1*p_160152960*t_2*l_2*p_2386560*l_2*p_53795926*t_2*l_1*p_1499520*l_2*p_127992597*t_1*l_1*p_2640000*l_1*p_85370418*l_1*p_11658240*t_1*l_2*p_1394880*l_1*p_465840*l_2*p_3784320>"),
MM("M<y_503h*x_9eah*d_374h*p_233989239*l_2*p_2386560*l_2*p_23198721*l_2*t_1*l_2*p_1900800*l_2*p_22760055*l_1*t_1*l_2*p_1858560*l_1*p_10668720*l_1*p_1506240*t_1*l_1*p_2640000*l_1*p_570627*t_2*l_2*p_58143360*l_2*p_168579888*l_1*t_2>"),
MM("M<y_0e5h*x_1f3ah*d_6c9h*p_90562686*l_2*p_1943040*l_2*p_22779366*l_1*t_1*l_2*p_1457280*l_1*p_10708*l_2*t_1*l_2*p_1457280*l_1*p_21801795*l_1*t_1*l_2*p_169920*l_1*t_1*l_1*p_1499520*l_2*p_86283409*t_2*l_2*p_1985280*l_1*p_85840899*t_2*l_2*p_1457280*l_1*p_533034*t_1*l_2*p_2830080*l_2*p_42667401>"),
MM("M<y_4d7h*x_0a6eh*d_0c5ah*p_80509897*l_2*p_2386560*l_2*p_31998076*l_1*t_2*l_1*p_1499520*l_1*p_21817250*l_1*t_1*l_2*p_1985280*l_1*p_32954433*l_2*t_2*l_2*p_1457280*l_1*p_4822*t_2*l_2*p_1457280*l_1*p_2885*t_2*l_2*p_2830080*l_2*p_10702299*l_2*t_2*l_2*p_78101760*l_2*p_168136464>"),
MM("M<y_5c2h*x_0f48h*d_43bh*p_184125088*l_2*p_2344320*l_2*p_32552120*l_1*t_2*l_2*p_2386560*l_2*p_32961161*t_2*l_1*p_467520*l_1*p_951552*t_2*l_2*p_1900800*l_2*p_151143*t_1*l_2*p_1457280*l_1*p_32466467*l_1*t_1*l_2*p_2956800*l_1*p_106660342*t_2*l_2*p_2597760*l_1*p_42717490>"),
MM("M<y_136h*x_1a2dh*d_0f15h*p_106908922*l_2*p_1985280*l_1*p_53353282*t_1*l_2*p_2787840*l_2*p_32511715*l_2*t_1*l_1*p_23040*l_1*p_2370816*t_1*l_2*p_2787840*l_2*p_86282450*l_2*p_528000*t_2*l_2*p_59917440*l_1*p_152169888>"),
MM("M<y_158h*x_26fh*d_0f44h*p_175760587*l_2*p_2344320*l_2*p_32070138*l_1*t_2*l_2*p_24000*l_2*p_10665840*l_2*t_2*l_1*p_1920*l_2*p_24336*l_2*p_2556480*t_2*l_2*p_2899200*l_2*t_1*l_2*p_2386560*l_2*p_42676067*t_2*l_1*p_1457280*l_2*p_96478695*l_1*p_2880*t_1*l_2*p_58143360*l_2*p_241317120>"),
MM("M<y_472h*x_19cdh*d_397h*p_96244732*l_2*p_2830080*l_2*p_2418726*l_2*t_1*l_2*p_2956800*l_1*p_23232*l_1*t_1*l_2*p_1900800*l_2*p_2789126*l_1*t_2*l_1*p_131520*l_2*t_2*l_2*p_2956800*l_1*p_11266023*t_2*l_2*p_1943040*l_2*p_42673189*t_1*l_2*p_2956800*l_1*p_42831938*t_2*l_2*p_2830080*l_2*p_43180260>")]

all([is_normalised(y6*x6, h) for h in y6x6_normaliser])

True

Elements normalising $g_6 = y_6x_6^2$.

In [23]:
y6x62_normaliser = [MM("M<y_5e5h*x_1a8h*d_88h*p_158548295*l_2*p_2597760*l_1*p_42705293*t_2*l_2*p_2386560*l_2*p_12107843*l_2*t_2*l_1*p_1415040*l_1*p_10668768*l_1*p_514560*t_2*l_1*p_3338880*l_2*t_1*l_1*p_2640000*l_1*p_34689*l_1*t_1*l_1*p_2027520*l_1*p_1936*t_2*l_1*p_1499520*l_2*p_42708851>"),
MM("M<y_534h*x_1d6bh*d_0c5ch*p_5343566*l_2*p_2386560*l_2*p_33401737*l_2*t_1*l_1*p_1457280*l_2*p_10666763*l_2*t_2*l_2*p_1858560*l_2*p_23376*l_1*p_4205760*t_1*l_1*p_2640000*l_1*p_217442*t_1*l_2*p_2830080*l_2*p_22755377*l_2*t_2*l_2*p_1900800*l_2*p_3857*t_2*l_2*p_1943040*l_2*p_64025696>"),
MM("M<y_536h*x_388h*d_28fh*p_121968387*l_2*p_1457280*l_1*p_22367548*l_2*t_1*l_2*p_3840*l_1*p_464832*l_1*p_1964160*t_1*l_2*p_1943040*l_2*p_21866291*l_1*t_1*l_1*p_2027520*l_1*p_475284*t_1*l_2*p_2386560*l_2*p_64015158*t_1*l_2*p_2386560*l_2*p_42730938*t_1*l_1*p_2999040*l_1*p_176106>"),
MM("M<y_587h*x_9c4h*d_16fh*p_123045651*l_2*p_2956800*l_1*p_32018112*l_2*t_2*l_2*p_2344320*l_2*p_21865408*l_1*t_1*l_1*p_960*l_2*p_22272*l_2*p_998400*t_1*l_2*p_2597760*l_1*p_96477776*t_1*l_2*p_2386560*l_2*p_53382136*t_2*l_1*p_1499520*l_1*p_21383234*t_1*l_2*p_762240>"),
MM("M<y_549h*x_22fh*d_30dh*p_219373891*l_2*p_1457280*l_1*p_21888592*l_1*t_1*l_2*p_1985280*l_1*p_1489449*l_1*t_1*l_1*p_2999040*l_1*p_1395206*l_2*t_1*l_1*p_2027520*l_1*p_152980*t_2*l_2*p_2597760*l_1*p_53824704*t_2*l_2*p_2386560*l_2*p_53794965*t_1*l_1*p_2999040*l_1*p_5796>"),
MM("M<y_561h*x_1ac0h*d_2e1h*p_184737550*l_2*p_2787840*l_2*p_32971681*l_2*t_2*l_2*p_1943040*l_2*p_10805169*t_2*l_2*p_2830080*l_2*p_21345811*t_2*l_1*p_1499520*l_2*p_951568*t_2*l_2*p_2830080*l_2*p_53436236*t_2*l_1*p_4190400*l_1*t_1*l_2*p_2033280*l_1>"),
MM("M<y_53dh*x_1e0bh*d_0fcdh*p_243978502*l_1*p_2640000*l_1*p_21871266*l_1*t_2*l_1*p_1499520*l_1*p_12996051*t_2*l_2*p_1900800*l_2*p_6742*t_2*l_2*p_1900800*l_2*p_1016355*t_2*l_1*p_1858560*l_2*p_466800*l_1*p_4312320*t_2*l_1*p_1499520*l_2*p_64002663*t_1*l_1*p_1499520*l_1*p_64016089>"),
MM("M<y_4bch*x_196ah*d_0e32h*p_11572968*l_1*p_1499520*l_2*p_21796210*l_1*t_2*l_2*p_2830080*l_2*p_33414272*t_1*l_2*p_1985280*l_1*p_11158316*t_1*l_1*p_1499520*l_1*p_53801654*t_2*l_2*p_2597760*l_1*p_42711672*t_2*l_2*p_2597760*l_1*p_43634329*t_2*l_1*p_1499520*l_2*p_106663237>"),
MM("M<y_1e0h*x_1593h*d_5c4h*p_198886190*l_2*p_2830080*l_2*p_21817304*t_2*l_2*p_2344320*l_2*p_1523012*t_2*l_1*p_2999040*l_1*p_47170*t_1*l_1*p_951360*t_2*l_2*p_2597760*l_1*p_42754026*t_2*l_2*p_2386560*l_2*p_42835787*t_2*l_2*p_1943040*l_2*p_43594872>"),
MM("M<y_19bh*x_153ch*d_0d43h*p_118504558*l_2*p_2344320*l_2*p_33420998*l_1*t_2*l_1*p_1457280*l_2*p_33397762*l_2*t_2*l_1*p_3840*l_1*p_1296*l_1*p_10394880*t_2*l_1*p_1499520*l_2*p_22326208*t_2*l_2*p_2344320*l_2*p_1912610*l_1*t_1*l_2*p_1943040*l_2*p_43160087*t_2*l_2*p_2597760*l_1*p_85833248>")]

all([is_normalised(y6*x6**2, h) for h in y6x62_normaliser])

True

Elements normalising $g_6 = y_6x_6^3$.

In [24]:
y6x63_normaliser = [MM("M<y_6fh*x_1cedh*d_484h*p_125326484*l_1*p_1457280*l_2*p_2794915*l_1*t_1*l_1*p_1393920*l_2*p_3168*l_1*p_1944000*t_1*l_2*p_1457280*l_1*p_22356999*l_2*t_2*l_2*p_2597760*l_1*p_53443799*t_2*l_1*p_3840*l_2*p_22272*l_2*p_1484160*t_1*l_1*p_16874880*l_2*t_2*l_2*p_1943040*l_2*p_43198480>"),
MM("M<y_4abh*x_0cebh*d_709h*p_6736343*l_2*p_1900800*l_2*p_962066*t_1*l_1*p_2640000*l_1*p_2791184*l_1*t_1*l_1*p_1920*l_2*p_10665792*l_2*p_805440*t_1*l_2*p_2597760*l_1*p_21348683*t_2*l_1*p_1499520*l_1*p_53377339*t_2*l_2*p_2386560*l_2*p_96040852*t_1*l_2*p_2787840*l_2*p_18272>"),
MM("M<y_431h*x_9efh*d_0c8fh*p_212298532*l_1*p_2027520*l_1*p_2793088*l_2*t_2*l_2*p_1985280*l_1*p_12059808*l_1*t_2*l_1*p_2027520*l_1*p_32555960*l_1*t_2*l_1*p_2999040*l_1*p_11523*t_2*l_2*p_49716480*l_2*p_233333856*t_1*l_2*p_1415040*l_2*p_2256*l_1*p_298560*t_1*l_2*p_1943040*l_2*p_42726212*t_1*l_2*p_2597760*l_1*p_42708905>"),
MM("M<y_273h*x_1c47h*d_0a7ah*p_124228372*l_2*p_24000*l_2*p_149232*t_2*l_2*p_2830080*l_2*p_13002563*t_2*l_2*p_1900800*l_2*p_1016258*t_1*l_2*p_2830080*l_2*p_21331778*t_2*l_1*p_1499520*l_2*p_85332897*t_2*l_2*p_2956800*l_1*p_43261057*t_1*l_1*p_1566720>"),
MM("M<y_3b1h*x_106h*d_0b7dh*p_31257692*l_1*p_1457280*l_2*p_12578473*t_2*l_2*p_1985280*l_1*p_12069346*l_2*t_2*l_1*p_1499520*l_2*p_32000790*l_1*t_2*l_2*p_2597760*l_1*p_106698932*t_1*l_1*p_2640000*l_1*p_22370393*l_2*t_1*l_2*p_2787840*l_2*p_5771*t_1*l_1*p_1499520*l_2*p_85327136>"),
MM("M<y_0ebh*x_1339h*d_466h*p_184229543*l_2*p_2830080*l_2*p_23202562*l_1*t_2*l_2*p_2386560*l_2*p_22306055*l_1*t_2*l_2*p_1393920*l_2*p_1200*l_1*p_960000*t_2*l_2*p_1985280*l_1*p_64026658*t_2*l_2*p_1858560*l_2*p_22320*l_1*p_5239680*t_2*l_1*p_1499520*l_2*p_106662250*t_1*l_1*p_1499520*l_1*p_85336736>"),
MM("M<y_8ch*x_1fe0h*d_0d92h*p_225498305*l_2*p_2386560*l_2*p_21931685*t_2*l_2*p_2787840*l_2*p_8738*l_1*t_2*l_2*p_2787840*l_2*p_32064452*l_1*t_1*l_2*p_4012800*l_1*t_1*l_2*p_1943040*l_2*p_42834857*t_2*l_2*p_2597760*l_1*p_21429419*t_2*l_1*p_2640000*l_1*p_63777*t_2*l_1*p_2027520*l_1*p_4816>"),
MM("M<y_5ech*x_104h*d_0d43h*p_238213356*l_1*p_59473920*l_2*p_203174400*t_2*l_1*p_80762880*l_2*p_183216000*l_1*t_1*l_2*p_70118400*l_2*p_220915200*l_1*t_1*l_2*p_2344320*l_2*p_5777*t_2*l_2*p_2956800*l_1*p_127989713*t_1*l_1*p_1457280*l_2*p_42755906*l_1*p_22080*t_1*l_1*p_79875840*l_1*p_2682240*l_2>"),
MM("M<y_121h*x_0b4ch*d_0f29h*p_208496975*l_2*p_1900800*l_2*p_1971435*l_1*t_1*l_1*p_2027520*l_1*p_12153188*l_1*t_2*l_2*p_2787840*l_2*p_21906706*l_1*t_1*l_1*p_1499520*l_1*p_21358224*t_2*l_1*p_1499520*l_1*p_106700754*t_2*l_2*p_2956800*l_1*p_21364050*t_2*l_1*p_4671360*l_1>"),
MM("M<y_56h*x_187dh*d_760h*p_214410543*l_2*p_2956800*l_1*p_32473189*l_1*t_2*l_1*p_2999040*l_1*p_32065424*l_1*t_1*l_2*p_2956800*l_1*p_23216165*l_1*t_2*l_2*p_2597760*l_1*p_11242963*t_1*l_2*p_1985280*l_1*p_106664193*t_1*l_2*p_1943040*l_2*p_11308374*t_2*l_1*p_1499520*l_1*p_42713669>")]

all([is_normalised(y6*x6**3, h) for h in y6x63_normaliser])

True

Elements normalising $g_6 = y_6x_3$.

In [25]:
y6x3_normaliser = [MM("M<y_469h*x_281h*d_0d74h*p_37530473*l_2*p_2597760*l_1*p_931392*t_1*l_1*p_22080*l_1*p_1858752*l_1*t_1*l_2*p_1394880*l_1*p_464976*l_2*p_93120*t_1*l_1*p_13326720*l_1*t_2*l_1*p_38400*l_2*t_2*l_1*p_3840*l_2*p_10665936*l_2*p_1949760*t_1*l_2*p_3125760*l_2*t_2*l_2*p_1985280*l_1*p_42728980>"),
MM("M<y_1bh*x_109eh*d_0ee3h*p_87372741*l_1*p_5136000*l_1*t_1*l_1*p_2027520*l_1*p_2369729*l_1*t_2*l_1*p_1499520*l_2*p_32006548*l_1*t_2*l_2*p_1943040*l_2*p_21933601*t_1*l_1*p_1499520*l_1*p_43602662*t_1*l_2*p_1943040*l_2*p_85416578*t_2*l_1*p_2027520*l_1*p_77025>"),
MM("M<y_0f5h*x_10a2h*d_0ebh*p_19705809*l_1*p_2027520*l_1*p_11658530*l_1*t_1*l_2*p_2597760*l_1*p_32972660*l_2*t_1*l_1*p_3840*l_1*p_22791936*l_1*t_1*l_2*p_1858560*l_2*p_53793024*t_1*l_2*p_1985280*l_1*p_43150564*t_1*l_2*p_1900800*l_2*p_597477*t_2*l_2*p_2830080*l_2*p_42757841>"),
MM("M<y_435h*x_1ad9h*d_0af8h*p_166320056*l_2*p_1457280*l_1*p_21416310*t_1*l_1*p_23040*l_1*p_32503968*l_1*t_1*l_2*p_2386560*l_2*p_22332049*l_1*t_1*l_1*p_4668480*l_1*t_1*l_1*p_2999040*l_1*p_132864*t_2*l_2*p_2597760*l_1*p_127993574*t_2*l_2*p_2386560*l_2*p_53351632*t_1*l_1*p_1499520*l_2*p_43638182>"),
MM("M<y_53h*x_1ddah*d_95bh*p_162032308*l_1*p_1457280*l_2*p_10670584*t_1*l_1*p_2640000*l_1*p_22753287*l_1*t_1*l_1*p_1415040*l_1*p_10668768*l_1*p_946560*t_1*l_2*p_2787840*l_2*p_15376*t_1*l_1*p_2640000*l_1*p_64121873*l_2*t_1*l_1*p_1499520*l_2*p_32125956*l_2>"),
MM("M<y_57dh*x_0e5fh*d_0e69h*p_3568471*l_2*p_2880*l_2*p_1394112*t_2*l_1*p_1499520*l_2*p_12557129*l_2*t_2*l_2*p_2344320*l_2*p_2395651*l_2*t_1*l_1*p_1499520*l_1*p_42676998*t_2*l_1*p_16431360*l_1*p_43507200*t_1*l_1*p_464640*l_2*p_43150464*t_2*l_2*p_1900800*l_2*p_529185*t_2*l_2*p_2386560*l_2*p_86257573>"),
MM("M<y_1c8h*x_4d1h*d_55fh*p_228244583*l_2*p_1985280*l_1*p_22815956*l_2*t_1*l_1*p_1457280*l_2*p_13052650*l_1*t_1*l_1*p_1499520*l_2*p_32113445*l_2*t_2*l_2*p_2956800*l_1*p_42677954*t_1*l_2*p_2830080*l_2*p_42664522*t_2*l_2*p_1943040*l_2*p_64042980*t_1*l_2*p_2386560*l_2*p_53355174*t_1*l_2*p_50603520*l_2*p_191199504>"),
MM("M<y_28fh*x_0d03h*d_0a9fh*p_20191243*l_1*p_2027520*l_1*p_1480819*t_1*l_1*p_2999040*l_1*p_29861*l_2*t_1*l_1*p_2999040*l_1*p_11669304*l_1*t_2*l_2*p_1457280*l_1*p_8673*t_1*l_1*p_79432320*l_1*p_240873888*t_1*l_1*p_2640000*l_1*p_573536*l_1*t_2*l_2*p_2880*l_2*p_464880*l_2*p_96000>"),
MM("M<y_155h*x_157bh*d_623h*p_118018712*l_1*p_1499520*l_2*p_13056521*t_2*l_2*p_2386560*l_2*p_21797111*l_1*t_1*l_2*p_1900800*l_2*p_22760119*l_2*t_1*l_2*p_1457280*l_1*p_10594*t_2*l_1*p_1499520*l_2*p_53437092*t_1*l_2*p_2956800*l_1*p_64001568*t_1*l_2*p_1900800*l_2*p_943858*t_2*l_1*p_1499520*l_1*p_106661289>"),
MM("M<y_50h*x_269h*d_960h*p_144165067*l_2*p_1943040*l_2*p_32959207*t_1*l_2*p_1457280*l_1*p_22778436*l_2*t_1*l_1*p_1858560*l_1*p_10668720*l_1*p_941760*t_1*l_2*p_1985280*l_1*p_64036417*t_1*l_2*p_2597760*l_1*p_64047746*t_2*l_2*p_2344320*l_2*p_127368*t_2*l_2*p_1985280*l_1*p_42671234>"),
MM("M<y_501h*x_0b88h*d_3fbh*p_54679531*l_2*p_1985280*l_1*p_466929*l_2*t_1*l_1*p_1457280*l_2*p_32995814*l_1*t_1*l_2*p_2787840*l_2*p_473394*l_2*t_1*l_1*p_69674880*l_2*p_220915200*t_1*l_2*p_2386560*l_2*p_85330964*t_2*l_2*p_1943040*l_2*p_21542978*t_2*l_2*p_2597760*l_1*p_21426656>"),
MM("M<y_82h*x_1ca6h*d_580h*p_42261164*l_2*p_1985280*l_1*p_32535011*l_1*t_1*l_2*p_1920*l_1*p_22272*l_1*p_1080960*t_2*l_2*p_1943040*l_2*p_32994842*l_1*t_2*l_2*p_2830080*l_2*p_64000641*t_1*l_2*p_1394880*l_1*p_466896*l_2*p_1954560*t_1*l_1*p_59030400*l_1*t_1*l_2*p_1985280*l_1*p_53793030>")]

all([is_normalised(y6*x3, h) for h in y6x3_normaliser])

True

### Proof of Proposition 3.5

The elements from Listing 6.

In [26]:
g13 = MM("M<y_519h*x_0cb8h*d_3abh*p_178084032*l_2*p_2344320*l_2*p_471482*l_1*t_1*l_2*p_2830080*l_2*p_22371347*l_2*t_2*l_1*p_1499520*l_2*p_22779365*l_2*t_1*l_2*p_2597760*l_1*p_11179396*t_1*l_1*p_1499520*l_2*p_85838017*t_2*l_1*p_1499520*l_1*p_64024721*t_2*l_2*p_2386560*l_2*p_21335269>")
g6 = MM("M<y_764h*x_590h*d_0bf6h*p_63465756*l_1*p_24000*l_2*p_528432*t_1*l_2*p_1457280*l_1*p_23214136*l_1*t_2*l_2*p_2344320*l_2*p_13038217*l_2*t_1*l_2*p_2956800*l_1*p_85332887*t_2*l_2*p_2830080*l_2*p_85335745*t_2*l_2*p_1900800*l_2*p_13472*t_2*l_2*p_2386560*l_2*p_85413728*t_1*l_2*p_2386560*l_2*p_53803593>")
i2 = MM("M<y_6ch*x_7ch*d_52ah*p_115885662*l_2*p_2787840*l_2*p_12552610*l_2*t_1*l_2*p_1900800*l_2*p_31998118*l_2*t_2*l_2*p_80762880*l_1*p_243091248*l_2*t_1*l_2*p_2597760*l_1*p_42794439*t_1*l_1*p_1394880*l_2*p_64015152*t_1*l_1*p_2027520*l_1*p_177984*t_1*l_2*p_79432320*l_1*p_161927136>")
a12 = MM("M<y_1afh*x_1661h*d_2ddh*p_208095583*l_2*p_1943040*l_2*p_1974295*l_2*t_2*l_2*p_1900800*l_2*p_10778*l_2*t_2*l_2*p_1900800*l_2*p_1868387*l_1*t_1*l_2*p_2956800*l_1*p_11159238*t_1*l_2*p_1985280*l_1*p_86275805*t_2*l_2*p_2386560*l_2*p_42712609*t_2*l_1*p_1499520*l_1*p_106699812>")

Check the presentation for $\mathrm{PSL}_2(13){:}2$.

In [27]:
u, v = i2*(g6**4)*a12, g13
u**2 == v**13 == (u*v**2)**4 == (u*v*u*v**4)**2 == MM("M<1>")

True

In [28]:
# Delete the variables u and v
del u, v

Confirm that $i_2$ satisfies the required properties of an involution "$j_2$", i.e. $i_2 \in 2B$ and inverts $g_6$.

In [29]:
i2.conjugate_involution()[0] == 2 and g6**i2 == g6**-1

True

Check that $\langle g_{13},g_6,i_2 \rangle \cong \mathrm{PSL}_2(13)$. As explained in the paper, at this point it sufficies to check that this group has order $1092$.

***This takes about a minute.***

In [30]:
group_generated_by([g13,g6,i2], 10000, order_only=True)

1092

Check that $i_2$ commutes with only the identity element of $\langle x_6 \rangle$, where $x_6$ is defined under "Proof of Proposition 3.1" above. As explained in the paper, this confirms that our $\mathrm{PSL}_2(13){:}2$ has trivial centraliser in $\mathrm{M}$.

In [31]:
[Comm(i2,x6**k) == MM("M<1>") for k in range(x6.order())].count(True)

1

Check class fusion. (Per the proof, only certain classes need to be checked computationally.)

First construct the group $G = \langle g_{13},g_6,i_2,a_{13} \rangle \cong \mathrm{PSL}_2(13){:}2$.

***This takes 2-3 minutes.***

In [32]:
G = group_generated_by([g13,g6,i2,a12], 10000)

Confirm that $|G| = 2184$.

In [33]:
len(G)

2184

Construct the element $g_{14} = a_{12}i_2g_{13}^2$ of order $14$, conjugate its square $g_7$ into $\mathrm{G}$ (and check containment), and check that $\chi_\mathrm{M}(g_7) = 1$ so that $g_7 \in 7\text{B}$.

In [34]:
g14 = a12*i2*g13**2
g7 = g14**2
g14.order() == 14 and (g7).order() == 7 and (g14**7).conjugate_involution()[0] == 2 and (g7**((g14**7).conjugate_involution()[1])).chi_G_x0()[0] == 1

True

Check that $a_{12}$ has order $12$, conjugate it into $\mathrm{G}$, and check that $\chi_\mathrm{M}(a_{12}) = 13$, so that $a_{12} \in 12\text{H}$.

In [35]:
a12.order() == 12 and (a12**((a12**6).conjugate_involution()[1])).in_G_x0() and (a12**((a12**6).conjugate_involution()[1])).chi_G_x0()[0] == 13

True

## Code accompanying Section 4

### Proof of Proposition 4.1

The elements from Listing 7.

In [36]:
x3 = MM("M<y_31h*x_0d92h*d_85ah*p_240874113*l_1*p_80762880*l_1*p_221802288*t_1*l_2*p_50160000*l_1*p_232003248*l_2*t_2*l_1*p_78988800*l_1*p_182328960*l_1*t_1*l_2*p_118018560*l_1*t_1*l_1*p_183216000*l_1>")
x10 = MM("M<y_491h*x_18h*d_77ah*p_179668320*l_1*p_68344320*l_2*p_159709440*l_2*t_1*l_1*p_70561920*l_2*p_242647728*l_2*t_1*l_1*p_79875840*l_1*p_182772480*l_1*t_1*l_1*p_4012800*l_2*t_1*l_2*p_117575040*l_1>")
a2 = MM("M<y_511h*x_19e5h*d_0f88h*p_175676956*l_2*p_127776000*t_2*l_1*p_60360960*l_1*p_193416960*l_2*t_1*l_1*p_69231360*l_2*p_162370608*l_2*t_2*l_1*p_67457280>")
a3 = MM("M<y_411h*x_158eh*d_64fh*p_160702030*l_2*p_1900800*l_2*p_684131*t_1*l_1*p_1499520*l_1*p_32064306*l_2*t_1*l_2*p_1394880*l_1*p_22320*l_2*p_98880*t_2*l_2*p_2830080*l_2*p_21469865*t_2*l_2*p_2830080*l_2*p_106661290*t_1*l_2*p_2597760*l_1*p_43613421*t_2*l_2*p_2830080*l_2*p_96456578>")
b2 = MM("M<y_599h*x_41ah*d_6b7h*p_240430467*l_1*p_70561920*l_1*p_140194560*t_1*l_1*p_81206400*l_2*p_169023408*l_1*t_1*l_2*p_79432320*l_2*p_212044848*l_2*t_1*l_2*p_59917440*l_1*p_157048416>")
b3 = MM("M<y_1eeh*x_15e7h*d_0d65h*p_141989494*l_1*p_59473920*l_2*p_131767728*l_2*t_2*l_2*p_50160000*l_2*p_179224368*l_2*t_2*l_1*p_71005440*l_1*p_243091248*l_1*t_2*l_1*p_58143360*l_2*p_179667936>")

Check that $x_3$ and $x_{10}$ satisfy the given presentation for $\mathrm{A}_{12}$.

In [37]:
x3**3 == x10**10 == (x3*x10)**11 == Comm(x3,x10)**2 == (x3*x10**-2*x3*x10**2)**2 == Comm(x3,x10**3)**2 == (x3*x10**-4*x3*x10**4)**2 == Comm(x3,x10**5)**2 == MM("M<1>")

True

Check that $\{a_2,a_3\}$ and $\{b_2,b_3\}$ satisfy the given presentation for $\mathrm{A}_5$. (The latter is needed in the proof of Proposition 4.2.)

In [38]:
a2**2 == a3**3 == (a2*a3)**5 == MM("M<1>") and b2**2 == b3**3 == (b2*b3)**5 == MM("M<1>")

True

### Proof of Proposition 4.2

Here we check that the type G, T, and B subgroups of $\mathbf{M}$ isomorphic to $\mathrm{A}_5$ have the claimed types. 

Various generator names are re-used in the paper, but here we add suffixes to indicate which type is being dealt with.

#### Type G

The relevant elements from Listing 8, "type G". (Some elements are not needed for this proof.)

Recall that $c_2$ is the central involution in $\mathbf{G} \cong 2^{1+24}.\mathrm{Co}_1$.

In [39]:
# type G
g2_G = MM("M<y_4f6h*x_1f98h*d_0b7h*p_67615847*l_1*p_2999040*l_1*p_86264262*l_2*p_11172480>")
g3_G = MM("M<y_4e1h*x_19cbh*d_9c8h*p_19643307*l_1*p_2999040*l_1*p_64003504*l_2*p_1478400>")
g5_G = g2_G*g3_G
c2_G = MM("M<x_1000h>")
c5_G = MM("M<y_548h*x_34ah*d_0a9ch*p_243281095*l_1*p_1457280*l_2*p_43255315*t_2*l_1*p_3840*l_2*p_465936*l_2*p_1101120*t_1*l_2*p_2787840*l_2*p_32009429*l_1*t_2*l_2*p_2956800*l_1*p_64018007*t_1*l_2*p_2880*l_2*p_3120*l_2*p_2579520*t_2*l_2*p_2830080*l_2*p_42706069*t_1*l_2*p_2787840*l_2*p_148289>")

Confirm that $\langle g_2,g_3 \rangle$ satisfy the presentation $g_2^2=g_3^3=(g_2g_3)^5=1$ for $\mathrm{A}_5$.

In [40]:
g2_G**2 == g3_G**3 == (g2_G*g3_G)**5 == MM("M<1>")

True

Check that $g_2$ and $g_3$ lie in $\mathbf{G}$ (equivalently, that they commute with $c_2$).

In [41]:
g2_G.in_G_x0() and g3_G.in_G_x0()

True

Check that $g_2 \in 2\text{B}$, that $g_3 \in 3\text{C}$ via $\chi_\mathrm{M}$-value $-1$, and that $g_5 = g_2g_3 \in 5\text{B}$ via $\chi_\mathrm{M}$-value $8$.

In [42]:
g2_G.conjugate_involution()[0] == 2 and g3_G.chi_G_x0()[0] == -1 and g5_G.chi_G_x0()[0] == 8

True

#### Type T

The elements from Listing 8, "type T".

In [43]:
g2_T = MM("M<y_82h*x_140eh*d_327h*p_130881367*l_1*p_80319360*l_1*p_131324208*l_1*t_1*l_1*p_69674880*l_2*p_160152960*l_1*t_1*l_1*p_48829440*l_1*p_230229120*l_2*t_1*l_1*p_70561920*l_1*p_87859296>")
g3_T = MM("M<y_430h*x_0d4h*d_8a2h*p_242204766*l_2*p_60804480*l_2*p_11552640*l_2*t_1*l_2*p_49272960*l_1*p_172128000*l_2*t_1*l_1*p_59917440*l_1*p_239986560*l_1*t_2*l_2*p_3125760*l_2*t_1*l_2*p_47055360>")
g5_T = g2_T*g3_T
c2_T = MM("M<d_200h>")
c3_T = MM("M<y_4cdh*x_1274h*d_499h*p_8151915*l_2*p_1900800*l_2*p_43255347*t_2*l_2*p_2597760*l_1*p_479249*l_2*t_2*l_1*p_4654080*t_1*l_2*p_2956800*l_1*p_53436116*t_2*l_2*p_2386560*l_2*p_85412773*t_1*l_1*p_1499520*l_1*p_106661296>")
i2_T = MM("M<y_1d9h*x_1d53h*d_170h*p_157936168*l_2*p_68344320*l_2*p_202730880*l_2*t_1*l_1*p_78545280*l_1*p_212044848*l_2*t_2*l_1*p_80762880*l_2*p_149508480*l_2*t_1*l_1*p_81206400*l_1*p_85198176>")
h_T = MM("M<y_17eh*x_143ah*d_0c93h*p_48068830*l_2*p_2956800*l_1*p_43160055*t_2*l_2*p_1943040*l_2*p_1471043*l_1*t_2*l_1*p_1499520*l_1*p_32513830*l_1*t_1*l_2*p_2830080*l_2*p_85329986*t_2*l_2*p_1985280*l_1*p_96485399*t_1*l_2*p_2386560*l_2*p_85330945>")

Confirm that $\langle g_2,g_3 \rangle$ satisfy the presentation $g_2^2=g_3^3=(g_2g_3)^5=1$ for $\mathrm{A}_5$.

In [44]:
g2_T**2 == g3_T**3 == (g2_T*g3_T)**5 == MM("M<1>")

True

Check that $g_2 \in 2\text{B}$, that $i_2 \in 2\text{B}$, and that $i_2$ centralises $g_5$ but not $g_2$ or $g_3$.

In [45]:
g2_T.conjugate_involution()[0] == 2 and i2_T.conjugate_involution()[0] == 2 and Comm(i2_T,g5_T) == MM("M<1>") and not(Comm(i2_T,g2_T) == MM("M<1>")) and not(Comm(i2_T,g3_T) == MM("M<1>"))

True

Check that $h$ conjugates $i_2$ to the central involution in $\mathrm{G}$ and conjugates $g_5$ into $\mathrm{G}$, and that $g_5 \in 5\text{B}$ via $\chi_\mathrm{M}(g_5)=8$.

In [46]:
i2_T**h_T == MM("M<x_1000h>") and (g5_T**h_T).in_G_x0() and (g5_T**h_T).chi_G_x0()[0] == 8

True

Check that $c_2$ and $c_3$ centralise $\langle g_2,g_3 \rangle$.

In [47]:
Comm(c2_T,g2_T) == Comm(c2_T,g3_T) == Comm(c3_T,g2_T) == Comm(c3_T,g3_T) == MM("M<1>")

True

#### Type B

The elements from Listing 8, "type B". Note that $g_2$ and $g_3$ are defined in terms of the elements $a_2$, $a_3$, $b_2$, and $b_3$ from Listing 7 (see above).

In [48]:
g2_B, g3_B = a2*b2, a3*b3
g5_B = g2_B*g3_B
c2_B = MM("M<y_15h*x_1c83h*d_955h*p_191219869*l_2*p_48829440*l_2*p_85198080*t_1*l_2*p_7560960*l_2*p_1795200*t_2*l_1*p_67013760*l_1>")
i2_B = MM("M<y_487h*x_1426h*d_602h*p_173036153*l_2*p_47055360*l_1*p_53264640*t_1*l_1*p_60360960*l_1*p_182772480*l_2*t_1*l_1*p_59473920*l_2*p_192086400*l_2*t_1*l_1*p_3569280*l_2*t_2*l_2*p_66570240*l_1>")
h_B = MM("M<y_4f1h*x_9bch*d_0f77h*p_106507260*l_1*p_80762880*l_2*p_213375504*t_2*l_1*p_1499520*l_2*p_583047*t_2*l_2*p_1900800*l_2*p_1040998*t_2*l_2*p_2386560*l_2*p_21331401*t_1>")

Confirm that $\langle g_2,g_3 \rangle$ satisfy the presentation $g_2^2=g_3^3=(g_2g_3)^5=1$ for $\mathrm{A}_5$.

In [49]:
g2_B**2 == g3_B**3 == (g2_B*g3_B)**5 == MM("M<1>")

True

Check that $g_2 \in 2\text{B}$, that $i_2 \in 2\text{B}$, and that $i_2$ centralises $g_5$ but not $g_2$ or $g_3$.

In [50]:
g2_B.conjugate_involution()[0] == 2 and i2_B.conjugate_involution()[0] == 2 and Comm(i2_B,g5_B) == MM("M<1>") and not(Comm(i2_B,g2_B) == MM("M<1>")) and not(Comm(i2_B,g3_B) == MM("M<1>"))

True

Check that $h$ conjugates $i_2$ to the central involution in $\mathrm{G}$ and conjugates $g_5$ into $\mathrm{G}$, and that $g_5 \in 5\text{B}$ via $\chi_\mathrm{M}(g_5)=8$.

In [51]:
i2_B**h_B == MM("M<x_1000h>") and (g5_B**h_B).in_G_x0() and (g5_B**h_B).chi_G_x0()[0] == 8

True

Check that $c_2 \in 2\text{A}$ and centralises $\langle g_2,g_3 \rangle$.

In [52]:
c2_B.conjugate_involution()[0] == 1 and Comm(c2_B,g2_B) == Comm(c2_B,g3_B) == MM("M<1>")

True

It remains to check that when regarded as a subgroup of $\langle x_3,x_{10} \rangle \cong \mathrm{A}_{12}$, the group $\langle b_2,b_3 \rangle$ has the claimed orbit lengths on $\Omega = \{1,\dots,12\}$.

The straight-line programs (SLPs) for obtaining $b_2$ and $b_3$ from $x_3$ and $x_{10}$ (and check).

In [53]:
def A5_in_A12_gen_1(a,b): b = a**2 * b; w3 = b * a; w4 = b * w3; w5 = a * w3; w6 = w4 * w5; w10 = w5 * a; w11 = w6 * w10; w16 = a**2; w17 = w11 * w16; w12 = b**-4; w7 = b**-3; w8 = w6 * w7; w9 = w8**-1; w13 = w12 * w9; w18 = w17 * w13; w19 = w18**-1; w20 = w10 * w16; w21 = w17 * w20; w22 = b**-5; w23 = w22 * w9; w24 = w21 * w23; w26 = w19 * w24; w14 = w11 * w13; w27 = w26 * w14; w28 = w10 * a; w29 = w21 * w28; w30 = w20 * a; w31 = w29 * w30; w39 = a * w28; w40 = w31 * w39; w49 = w39 * w28; w50 = w40 * w49; w55 = w39 * w30; w56 = w50 * w55; w32 = b**3; w33 = a * w19; w15 = w14**-1; w34 = w15 * w9; w35 = w33 * w34; w36 = w32 * w35; w37 = w31 * w36; w57 = w37**-2; w58 = a * w57; w59 = w58 * w9; w25 = w24**-1; w38 = w37**-1; w60 = w25 * w38; w44 = w9 * w38; w51 = w15 * w44; w52 = w33 * w51; w53 = w50 * w52; w54 = w53**-1; w41 = b**2; w42 = w41 * a; w43 = w19 * w15; w45 = w43 * w44; w46 = w42 * w45; w47 = w40 * w46; w48 = w47**-1; w61 = w54 * w48; w62 = w60 * w61; w63 = w59 * w62; w64 = w56 * w63; w66 = w27 * w64; w67 = w66 * w24; w68 = w67 * w14; w69 = w68 * w47; w70 = w49 * w39; w71 = w56 * w70; w77 = w55 * w49; w78 = w71 * w77; w1 = a**-1; w79 = w1 * w9; w80 = w19 * w34; w81 = w79 * w80; w82 = w19 * w38; w65 = w64**-1; w72 = w38 * w9; w73 = w65 * w60; w74 = w72 * w73; w75 = w71 * w74; w83 = w75**-2; w84 = w65 * w83; w85 = w82 * w84; w86 = w81 * w85; w87 = w78 * w86; w89 = w69 * w87; w90 = w89 * w87; w91 = w90 * w37; w92 = w91 * w18; return w92;
def A5_in_A12_gen_2(a,b): b = a**2 * b; w3 = b * a; w4 = b * w3; w5 = a * w3; w6 = w4 * w5; w10 = w5 * a; w11 = w6 * w10; w16 = a**2; w17 = w11 * w16; w20 = w10 * w16; w21 = w17 * w20; w28 = w10 * a; w29 = w21 * w28; w30 = w20 * a; w31 = w29 * w30; w39 = a * w28; w40 = w31 * w39; w49 = w39 * w28; w50 = w40 * w49; w55 = w39 * w30; w56 = w50 * w55; w70 = w49 * w39; w71 = w56 * w70; w77 = w55 * w49; w78 = w71 * w77; w93 = w55 * w70; w94 = w78 * w93; w22 = b**-5; w12 = b**-4; w7 = b**-3; w8 = w6 * w7; w9 = w8**-1; w13 = w12 * w9; w18 = w17 * w13; w19 = w18**-1; w14 = w11 * w13; w15 = w14**-1; w43 = w19 * w15; w95 = w22 * w43; w32 = b**3; w33 = a * w19; w34 = w15 * w9; w35 = w33 * w34; w36 = w32 * w35; w37 = w31 * w36; w38 = w37**-1; w57 = w37**-2; w58 = a * w57; w59 = w58 * w9; w23 = w22 * w9; w24 = w21 * w23; w25 = w24**-1; w60 = w25 * w38; w44 = w9 * w38; w51 = w15 * w44; w52 = w33 * w51; w53 = w50 * w52; w54 = w53**-1; w41 = b**2; w42 = w41 * a; w45 = w43 * w44; w46 = w42 * w45; w47 = w40 * w46; w48 = w47**-1; w61 = w54 * w48; w62 = w60 * w61; w63 = w59 * w62; w64 = w56 * w63; w65 = w64**-1; w96 = w38 * w65; w97 = w9 * w96; w72 = w38 * w9; w73 = w65 * w60; w74 = w72 * w73; w75 = w71 * w74; w83 = w75**-2; w1 = a**-1; w79 = w1 * w9; w80 = w19 * w34; w81 = w79 * w80; w82 = w19 * w38; w84 = w65 * w83; w85 = w82 * w84; w86 = w81 * w85; w87 = w78 * w86; w88 = w87**-1; w98 = w88 * w19; w99 = w83 * w98; w100 = w97 * w99; w101 = w95 * w100; w102 = w94 * w101; w103 = w102**-1; w104 = w103 * w18; w105 = w104 * w24; w106 = w105 * w64; w107 = w106 * w24; w108 = w107 * w14; w109 = w108 * w48; w110 = w109 * w88; w111 = w110 * w88; w112 = w111 * w38; w76 = w75**-1; w113 = w112 * w76; w114 = w113 * w8; w115 = w114 * w75; w116 = w115 * b; w117 = w116 * b; return w117;

In [54]:
A5_in_A12_gen_1(x3, x10) == b2 and A5_in_A12_gen_2(x3, x10) == b3

True

GAP versions of the same SLPs, and GAP code showing that the $\mathrm{A}_5 < \mathrm{A}_{12}$ constructed from these SLPs (as described in the paper) has orbits of length $6$ and $6$ on $\Omega$, can be found at:

https://github.com/melissa-maths/M/blob/main/A5_in_A12_SLP.g

(One can confirm that the mmgroup and GAP versions of the code define the same SLPs.)

## Code accompanying Section 5

In the fourth paragraph of Section 5, we refer to the character values of the elements $g_5$ from each of the cases "type G", "type T", and "type B" above in the $24$-dimensional representation of $\mathrm{Co}_1$ over $\mathbb{F}_2$. We claim that for types G and B, $g_5$ should have fixed-point space of dimension $4$; and that for type T, $g_5$ should have trivial fixed-point space. This corresponds to $g_5$ having $\chi_{24}$-value $-1$ for types G and T, and $-6$ for type B. The following code checks that this is indeed the case. Note that mmgroup calculates $\chi_{24}$ only up to sign, so the calculated character values are actually $1$ and $6$. Also recall that $g_5 \in \mathbf{G}$ only in "type G"; in the other two cases, we must first conjugate $g_5$ into $\mathbf{G}$ via the element "$h$" defined above.

In [55]:
g5_G.chi_G_x0()[2] == (g5_B**h_B).chi_G_x0()[2] == 1 and (g5_T**h_T).chi_G_x0()[2] == 6

True

The arguments in Section 5 also refer to elements generating certain 'large' subgroups $Y$ of the normalisers of the elements $g_5$ (for each of the three cases). These elements are as follows, with checks that they normalise the corresponding $g_5$. The involutions "$j_2$" inverting the $g_5$ can be found via random search in the groups generated by the corresponding elements (using e.g. the function get_random).

In [56]:
# Elements normalising g5_G
normaliser_of_g5_G = [MM("M<y_401h*x_0c4dh*d_193h*p_214145658*l_2*p_2880*l_2*p_10667856*l_1*p_2434560>"),
 MM("M<y_748h*x_19ah*d_671h*p_32272403*l_1*p_2027520*l_1*p_11240054*l_1>"),
 MM("M<y_7aeh*x_1530h*d_0eb9h*p_84540987*l_2*p_2830080*l_2*p_106666098*l_2*p_2344320>"),
 MM("M<y_0b2h*x_1f50h*d_166h*p_52605150*l_2*p_1394880*l_2*p_11659536*l_1>"),
 MM("M<x_861h*d_8cfh>"),
 MM("M<x_4feh*d_51bh>"),
 MM("M<x_28ch*d_4ceh>"),
 MM("M<x_1ebh*d_5edh>"),
 MM("M<y_442h*x_18b3h*d_5c1h*p_223189606*l_2*p_1985280*l_1*p_32794>"),
 MM("M<y_477h*x_0cdch*d_7fdh*p_85726267*l_1*p_2640000*l_1*p_86275793*t_2*l_2*p_1985280*l_1*p_12545571*l_2*t_1*l_1*p_171684480*t_2*l_1*p_1499520*l_1*p_106704594*t_1*l_2*p_1985280*l_1*p_53393670>"),
 MM("M<y_52dh*x_1851h*d_0b46h*p_191358785*l_2*p_1985280*l_1*p_21887385>"),
 MM("M<y_6eh*x_0f2h*d_83bh*p_146388894*l_1*p_1499520*l_1*p_32996736*l_1>"),
 MM("M<y_17h*x_717h*d_962h*p_122220815*l_2*p_1943040*l_2*p_33445043*l_1*t_2*l_2*p_2386560*l_2*p_21796112*l_1*t_2*l_2*p_1457280*l_1*p_22760119*l_1*t_2*l_2*p_2386560*l_2*p_42676035*t_2*l_2*p_2956800*l_1*p_106665162*t_1*l_2*p_2956800*l_1*p_85411814*t_1*l_2*p_1943040*l_2*p_21376531*t_1*l_1*p_1499520*l_2*p_21447718>"),
 MM("M<y_1a4h*x_68dh*d_270h*p_570798*l_2*p_1943040*l_2*p_53370860*t_1*l_1*p_2027520*l_1*p_1397931*l_2*t_1*l_1*p_172128000*t_2*l_2*p_1900800*l_2*p_8642*t_1*l_2*p_1943040*l_2*p_21440992>"),
 MM("M<y_4c2h*x_0c5ch*d_0eb4h*p_173437846*l_2*p_69674880*l_1*p_199626288*t_2*l_1*p_49272960*l_2*p_240873600*l_2*t_1*l_1*p_2999040*l_1*p_1948448*l_2*t_2*l_2*p_1900800*l_2*p_1056394*t_1*l_2*p_1985280*l_1*p_42728035*t_1*l_2*p_2830080*l_2*p_42712720*t_2*l_1*p_1499520*l_2*p_85332899>")]

# Check
all([is_normalised(g5_G,h) for h in normaliser_of_g5_G])

True

In [57]:
# Elements normalising g5_T
normaliser_of_g5_T = [MM("M<y_66h*x_1e9dh*d_0f2ah*p_137492110*l_2*p_1985280*l_1*p_12167510*t_2*l_1*p_3840*l_2*p_32461776*l_2*t_2*l_2*p_1415040*l_1*p_10666800*l_1*p_6066240*t_2*l_1*p_2821440*l_2*t_2*l_1*p_1499520*l_1*p_21333338*t_1*l_2*p_2830080*l_2*p_32948486*l_1*t_2*l_2*p_1943040*l_2*p_42730008*t_1*l_2*p_3316800>"),
 MM("M<y_1ebh*x_15cbh*d_3e8h*p_214705904*l_1*p_60360960*l_2*p_132211248*t_2*l_1*p_71005440*l_1*p_239986560*l_1*t_2*l_2*p_19092480*l_2*p_2238720*l_1*t_2*l_2*p_70561920*l_1*p_42620352>"),
 MM("M<y_118h*x_1a32h*d_8d4h*p_208898175*l_1*p_933120*l_2*t_2*l_1*p_1394880*l_2*p_2831520*l_2*t_2*l_2*p_2386560*l_2*p_2392784*l_1*t_2*l_1*p_2640000*l_1*p_6752*t_1*l_1*p_2027520*l_1*p_130930*t_1*l_2*p_1985280*l_1*p_64044903*t_1*l_2*p_1457280*l_1*p_178961*t_2*l_2*p_1943040*l_2*p_53803620>"),
 MM("M<y_51dh*x_160eh*d_0bc9h*p_200217151*l_2*p_1943040*l_2*p_473506*l_1*t_2*l_2*p_2956800*l_1*p_22456*l_1*t_2*l_2*p_2787840*l_2*p_22309079*l_2*t_2*l_2*p_1943040*l_2*p_127992615*t_1*l_1*p_1499520*l_1*p_42676994*t_1*l_1*p_2027520*l_1*p_54897*t_1*l_2*p_2386560*l_2*p_43622741>"),
 MM("M<y_5bh*x_1359h*d_0cc4h*p_190756722*l_2*p_1985280*l_1*p_32950456*t_1*l_2*p_2344320*l_2*p_12598582*l_2*t_1*l_1*p_2999040*l_1*p_11709585*l_2*t_2*l_2*p_2386560*l_2*p_11660533*t_1*l_1*p_2999040*l_1*p_96036066*l_2*p_21333120*t_1*l_1*p_960*l_2*p_10665840*l_2*p_1931520>"),
 MM("M<y_468h*x_16a5h*d_5f2h*p_99157995*l_2*p_2830080*l_2*p_1925313*l_2*t_1*l_2*p_2956800*l_1*p_1399859*l_2*t_1*l_2*p_1985280*l_1*p_22778418*l_1*t_1*l_2*p_1985280*l_1*p_85326147*t_2*l_2*p_2386560*l_2*p_85820731*t_2*l_2*p_1943040*l_2*p_96461344*t_2*l_1*p_2640000*l_1*p_51073*t_1*l_2*p_1985280*l_1*p_53805493>"),
 MM("M<y_45ah*x_18c0h*d_891h*p_138166856*l_2*p_2787840*l_2*p_21817157*l_2*t_2*l_2*p_1920*l_1*p_464928*l_1*p_2369280*t_2*l_2*p_1900800*l_2*p_1511601*l_2*t_2*l_1*p_1499520*l_2*p_10779194*t_2*l_1*p_68787840*l_1*p_210270720*l_2*t_1*l_2*p_173015040*l_2>"),
 MM("M<y_270h*x_47eh*d_0c32h*p_145938732*l_1*p_1499520*l_2*p_1486598*l_2*t_1*l_2*p_2597760*l_1*p_471569*l_2*t_1*l_1*p_2027520*l_1*p_12995026*t_1*l_2*p_1943040*l_2*p_53826660*t_1*l_1*p_1499520*l_1*p_21463125*t_2*l_1*p_1499520*l_2*p_42747518>"),
 MM("M<y_200h*x_15c2h*d_57dh*p_8511108*l_1*p_1499520*l_1*p_22774566*l_2*t_1*l_2*p_1900800*l_2*p_23338*l_1*t_1*l_2*p_2956800*l_1*p_23191028*l_1*t_2*l_1*p_1499520*l_1*p_13016293*t_1*l_2*p_2956800*l_1*p_42676034*t_2*l_2*p_2386560*l_2*p_53354273*t_2*l_1*p_1499520*l_2*p_42707940>"),
 MM("M<y_49eh*x_10b5h*d_0b9fh*p_105135069*l_2*p_59473920*l_2*p_63909312*t_2*l_2*p_2597760*l_1*p_10667802*l_2*t_1*l_2*p_1858560*l_1*p_465792*l_2*p_499200*t_2*l_2*p_1457280*l_1*p_1901124*t_2*l_2*p_2597760*l_1*p_85372329*t_1*l_1*p_1499520*l_2*p_42796354*t_2*l_1*p_3254400*l_2>"),
 MM("M<y_8ch*x_1484h*d_514h*p_30834449*l_2*p_2597760*l_1*p_43613397*t_1*l_2*p_1943040*l_2*p_13058419*l_2*t_2*l_1*p_199680*t_1*l_2*p_2830080*l_2*p_53823776*t_1*l_2*p_2830080*l_2*p_21347617*t_2*l_2*p_2386560*l_2*p_53907499>")]

# Check
all([is_normalised(g5_T,h) for h in normaliser_of_g5_T])

True

In [58]:
# Elements normalising g5_B
normaliser_of_g5_B = [MM("M<y_0b7h*x_8e8h*d_873h*p_125664661*l_2*p_1985280*l_1*p_572564*t_1*l_2*p_2956800*l_1*p_1471041*l_1*t_2*l_2*p_1985280*l_1*p_2818145*l_1*t_2*l_1*p_2999040*l_1*p_13464*t_1*l_1*p_1499520*l_1*p_85335765*t_2*l_1*p_1499520*l_1*p_42676036*t_2*l_2*p_2956800*l_1*p_85370401*t_2*l_1*p_78988800*l_2*p_161927136>"),
 MM("M<y_64ch*x_1b3eh*d_20h*p_164693784*l_1*p_2027520*l_1*p_22367553*t_1*l_2*p_2344320*l_2*p_10666777*l_1*t_1*l_1*p_1858560*l_1*p_465936*l_2*p_623040*t_2*l_2*p_2830080*l_2*p_21419906*t_1*l_1*p_1393920*l_2*p_21456*l_1*p_764160*t_1*l_2*p_1900800*l_2*p_1953*t_2*l_2*p_2386560*l_2*p_53443812>"),
 MM("M<y_4b2h*x_1de8h*d_4d5h*p_105600541*l_2*p_2830080*l_2*p_22310027*l_2*t_1*l_2*p_2830080*l_2*p_33454538*l_1*t_1*l_2*p_2787840*l_2*p_21796180*l_1*t_2*l_2*p_2597760*l_1*p_43174435*t_1*l_1*p_1499520*l_2*p_64128619*t_2*l_2*p_2830080*l_2*p_85327127*t_1*l_2*p_2956800*l_1*p_96478691>"),
 MM("M<y_141h*x_6ffh*d_87fh*p_50856855*l_1*p_2999040*l_1*p_12610118*t_1*l_2*p_2830080*l_2*p_25969*t_2*l_1*p_2027520*l_1*p_1940*t_1*l_1*p_1499520*l_2*p_11223699*t_1*l_1*p_1394880*l_1*p_10668768*l_2*p_132480*t_2*l_1*p_464640*l_2*p_42706032*t_1*l_1*p_1499520*l_1*p_21424627>"),
 MM("M<y_11bh*x_0cc7h*d_0b47h*p_194811739*l_2*p_1457280*l_1*p_33400789*t_1*l_2*p_2956800*l_1*p_13012305*l_2*t_2*l_2*p_1457280*l_1*p_21890512*l_1*t_1*l_2*p_2386560*l_2*p_42799222*t_2*l_2*p_50603520*l_1*p_173015040*l_1*t_2*l_1*p_47498880*l_2*p_240873888>"),
 MM("M<y_400h*x_1973h*d_156h*p_41501581*l_2*p_1943040*l_2*p_53821828*t_1*l_2*p_1900800*l_2*p_21805601*l_2*t_1*l_2*p_1985280*l_1*p_23218100*l_1*t_2*l_1*p_1499520*l_1*p_11160147*t_1*l_2*p_59473920*l_1*p_172128000*l_1*t_1*l_2*p_13770240*l_2>"),
 MM("M<y_4d1h*x_1174h*d_0d83h*p_131556556*l_1*p_1499520*l_1*p_21929794*l_1*t_2*l_2*p_1457280*l_1*p_131858*t_1*l_1*p_2999040*l_1*p_515664*t_2*l_1*p_2640000*l_1*p_217473*t_1*l_1*p_70561920*l_1*p_63909312*t_2*l_1*p_71005440*l_1*p_63909408>"),
 MM("M<y_563h*x_1dbfh*d_0d2h*p_203237214*l_2*p_2386560*l_2*p_1526983*l_1*t_2*l_2*p_18648960*l_1*p_12883200*l_2*t_2*l_2*p_2880*l_2*p_22368*l_2*p_963840*t_2*l_1*p_26880*l_2*t_1*l_2*p_71005440*l_2*p_85198080*t_1*l_1*p_55482240*l_2*p_51469440*l_1*t_2*l_1*p_70561920>"),
 MM("M<y_1ech*x_1576h*d_0ed7h*p_165495892*l_2*p_80762880*l_1*p_200956944*t_2*l_1*p_71005440*l_2*p_192973440*l_1*t_1*l_1*p_49272960*l_1*p_232890288*l_2*t_2*l_1*p_70561920>"),
 MM("M<y_12ch*x_1da3h*d_52eh*p_194452602*l_2*p_2956800*l_1*p_1529873*l_1*t_1*l_1*p_2999040*l_1*p_22307957*l_1*t_2*l_1*p_2640000*l_1*p_2353526*l_1*t_2*l_1*p_1499520*l_1*p_85328070*t_1*l_2*p_2597760*l_1*p_96480609*t_2*l_1*p_2640000*l_1*p_11141956*l_1*t_1*l_1*p_50603520*l_2*p_240873696>"),
 MM("M<y_0f5h*x_0efbh*d_5e9h*p_182899602*l_2*p_1457280*l_1*p_12596644*l_1*t_2*l_2*p_2830080*l_2*p_12176289*l_1*t_1*l_2*p_1393920*l_1*p_10668816*l_2*p_2473920*t_1*l_2*p_1985280*l_1*p_32000867*t_1*l_2*p_2830080*l_2*p_42838694*t_2*l_2*p_2597760*l_1*p_127993573*t_1*l_2*p_2787840*l_2*p_97152>"),
 MM("M<y_229h*x_1dc0h*d_0fe0h*p_231348152*l_1*p_1457280*l_2*p_1015203*t_1*l_2*p_1943040*l_2*p_31998026*l_1*t_1*l_2*p_2344320*l_2*p_2834359*l_2*t_1*l_2*p_1859520*l_2*t_1*l_1*p_1499520*l_1*p_106703659*t_1*l_2*p_2956800*l_1*p_85330963*t_1*l_2*p_2830080*l_2*p_11216307*t_1*l_2*p_1943040*l_2*p_127988771>"),
 MM("M<y_756h*x_1fh*d_20bh*p_6736627*l_1*p_1499520*l_2*p_11703830*l_1*t_2*l_1*p_2027520*l_1*p_3124*l_1*t_2*l_2*p_1394880*l_2*p_466848*l_1*p_3763200*t_1*l_2*p_2386560*l_2*p_23254528*t_2*l_2*p_1900800*l_2*p_22768694*l_1*t_1*l_2*p_2956800*l_1*p_42837733*t_1*l_1*p_1499520*l_2*p_64006378>"),
 MM("M<y_103h*x_4d9h*d_16ch*p_56537890*l_1*p_2640000*l_1*p_21888598*t_1*l_1*p_1394880*l_1*p_11616240*l_1*t_2*l_1*p_1394880*l_2*p_21360*l_1*p_107520*t_2*l_2*p_6720*l_1*t_1*l_2*p_2344320*l_2*p_65491*t_2*l_2*p_1985280*l_1*p_85331939*t_2*l_2*p_2956800*l_1*p_64017978*t_1*l_2*p_1943040*l_2*p_96461345>"),
 MM("M<y_41dh*x_19a7h*d_41bh*p_105177535*l_1*p_1499520*l_1*p_10688160*t_2*l_1*p_466560*l_1*p_1904160*l_2*t_1*l_2*p_2956800*l_1*p_2410055*l_2*t_1*l_2*p_1943040*l_2*p_42718449*t_1*l_2*p_2597760*l_1*p_96021712*t_2*l_2*p_1985280*l_1*p_63999749*t_1*l_1*p_2640000*l_1*p_14426>")]

# Check
all([is_normalised(g5_B,h) for h in normaliser_of_g5_B])

True

In the "type B" case, we assert that there are exactly 40 involutions that invert $g_5$ and extend $A_B \cong \mathrm{A}_5$ to a subgroup of $\mathbf{M}$ isomorphic to $\mathrm{PSL}_2(16)$ that does not extend to an almost simple maximal subgroup. Here are the claimed 40 involutions for reference.

In [59]:
involutions_inverting_g5_B = [MM("M<y_1a6h*x_1126h*d_4b0h*p_114554739*l_2*p_2344320*l_2*p_1913639*l_2*t_2*l_2*p_1920*l_1*p_3168*l_1*t_1*l_1*p_960*l_1*p_466848*l_1*p_2959680*t_1*l_2*p_2518080*l_2*t_1*l_1*p_1499520*l_2*p_21379397*t_2*l_2*p_2386560*l_2*p_96021731*t_2*l_2*p_2597760*l_1*p_64044905*t_2*l_2*p_2830080*l_2*p_43592085>"),
 MM("M<y_8ah*x_965h*d_0c4bh*p_222266216*l_2*p_50160000*l_2*p_220471680*l_2*t_2*l_1*p_50160000*l_2*p_233333856*t_1*l_1*p_1499520*l_2*p_43615126*t_2*l_2*p_2597760*l_1*p_53795810*t_2*l_1*p_1920*l_1*p_1152*l_1*p_298560*t_1*l_1*p_1499520*l_2*p_170654249*t_1*l_2*p_2597760*l_1*p_53459129>"),
 MM("M<y_452h*x_0eb7h*d_33ch*p_92907778*l_2*p_1985280*l_1*p_32509776*l_1*t_2*l_1*p_1499520*l_1*p_12991986*l_1*t_2*l_2*p_1415040*l_2*p_21312*l_1*p_3463680*t_1*l_2*p_2956800*l_1*p_64002534*t_2*l_2*p_1943040*l_2*p_42757851*t_2*l_2*p_2386560*l_2*p_85815926*t_2*l_2*p_2597760*l_1*p_86259459>"),
 MM("M<y_18fh*x_7f9h*d_0b4ch*p_58693155*l_2*p_2344320*l_2*p_21829666*l_1*t_2*l_2*p_2787840*l_2*p_23223812*l_1*t_1*l_1*p_2880*l_1*p_4032*l_1*p_616320*t_2*l_2*p_1985280*l_1*p_11616300*t_2*l_2*p_2830080*l_2*p_86280560*t_2*l_2*p_2956800*l_1*p_42718449*t_2*l_1*p_4266240>"),
 MM("M<y_155h*x_0cb4h*d_327h*p_77595109*l_1*p_4797120*t_1*l_2*p_2956800*l_1*p_2390900*l_1*t_2*l_2*p_1920*l_1*p_23280*l_1*p_1488960*t_2*l_2*p_49272960*l_2*p_230229120*t_2*l_2*p_1943040*l_2*p_86264260*l_1*p_22080*t_1*l_2*p_2830080*l_2*p_96480594*l_1*p_10665600>"),
 MM("M<y_155h*x_0d55h*d_3d1h*p_69209502*l_2*p_2597760*l_1*p_21333320*t_1*l_1*p_1499520*l_2*p_29864*l_2*t_2*l_2*p_2386560*l_2*p_12997012*l_1*t_2*l_2*p_2830080*l_2*p_127990698*t_2*l_1*p_2027520*l_1*p_494523*t_1*l_1*p_1499520*l_2*p_85377141*l_1*t_1*l_2*p_2956800*l_1*p_498417>"),
 MM("M<y_412h*x_85eh*d_6dch*p_226301460*l_2*p_1985280*l_1*p_528386*t_1*l_2*p_1393920*l_1*p_22416*l_2*t_1*l_1*p_1920*l_1*p_465888*l_1*p_993600*t_2*l_1*p_25920*l_1*t_2*l_1*p_1499520*l_1*p_10730147*t_2*l_2*p_1943040*l_2*p_42832931*t_2*l_2*p_1985280*l_1*p_10720487*t_2*l_2*p_2956800*l_1*p_43256258>"),
 MM("M<y_0f6h*x_1e81h*d_991h*p_147924974*l_2*p_1457280*l_1*p_12156080*l_1*t_2*l_2*p_2597760*l_1*p_23197937*l_1*t_1*l_1*p_2880*l_2*p_23232*l_2*p_4293120*t_2*l_2*p_2830080*l_2*p_21503536*t_2*l_1*p_1457280*l_2*p_11136163*l_2*t_2*l_2*p_1457280*l_1*p_32103844*l_1>"),
 MM("M<y_77h*x_0c1dh*d_33ah*p_118019516*l_2*p_2787840*l_2*p_42858*t_1*l_1*p_1499520*l_1*p_23262304*l_2*t_1*l_1*p_1394880*l_1*p_2160*l_2*p_2477760*t_2*l_2*p_2830080*l_2*p_11144810*t_1*l_2*p_2830080*l_2*p_42728066*t_1*l_1*p_1499520*l_2*p_53816981*t_1*l_1*p_1499520*l_1*p_64045846>"),
 MM("M<y_47fh*x_78fh*d_4efh*p_171705371*l_2*p_2830080*l_2*p_12110842*l_2*t_1*l_2*p_1394880*l_1*p_466896*l_2*p_130560*t_1*l_2*p_2787840*l_2*p_2801632*l_1*t_2*l_1*p_1499520*l_1*p_31998006*t_2*l_2*p_1985280*l_1*p_63994952*t_2*l_2*p_1457280*l_1*p_1016370*t_1*l_2*p_2787840*l_2*p_160673>"),
 MM("M<y_753h*x_21dh*d_0f00h*p_110056107*l_1*p_1920*l_2*p_971760*t_2*l_2*p_1943040*l_2*p_10666777*l_1*t_2*l_1*p_1393920*l_1*p_2256*l_2*p_179520*t_1*l_2*p_2597760*l_1*p_43263010*t_2*l_2*p_2386560*l_2*p_43706455*t_1*l_2*p_1985280*l_1*p_43678788*t_1*l_2*p_1985280*l_1*p_64005410>"),
 MM("M<y_51ah*x_528h*d_0bf3h*p_189869481*l_2*p_1985280*l_1*p_64036461*t_1*l_1*p_1457280*l_2*p_10698375*t_1*l_1*p_1499520*l_1*p_11188072*t_2*l_2*p_1457280*l_1*p_93337*t_1*l_2*p_1457280*l_1*p_219393*t_1*l_1*p_1457280*l_2*p_44546*t_1*l_2*p_1943040*l_2*p_53837191>"),
 MM("M<y_42dh*x_143eh*d_0a3ah*p_83487431*l_2*p_2344320*l_2*p_23264209*l_2*t_1*l_2*p_2956800*l_1*p_5940*l_1*t_2*l_2*p_2830080*l_2*p_22377108*l_1*t_1*l_2*p_2830080*l_2*p_12579257*t_1*l_1*p_2027520*l_1*p_106661314*l_2*p_10668480*t_1*l_2*p_2956800*l_1*p_1526054*l_1>"),
 MM("M<y_3h*x_80fh*d_0b85h*p_15206084*l_2*p_2344320*l_2*p_12597633*l_2*t_2*l_2*p_2386560*l_2*p_32549317*l_1*t_2*l_1*p_960*l_2*p_21456*l_2*p_2437440*t_1*l_1*p_1457280*l_2*p_70228*t_2*l_2*p_2344320*l_2*p_139498*t_1*l_1*p_1499520*l_2*p_85370411*t_2*l_2*p_1985280*l_1*p_64120979>"),
 MM("M<y_42bh*x_0d79h*d_0fd9h*p_118566642*l_1*p_466560*l_1*p_507360*t_2*l_2*p_1900800*l_2*p_12550530*l_1*t_2*l_2*p_1394880*l_2*p_24336*l_1*p_1521600*t_1*l_2*p_2386560*l_2*p_64001584*t_1*l_2*p_2597760*l_1*p_85336723*t_2*l_1*p_1499520*l_2*p_53902691*t_1*l_2*p_1985280*l_1*p_127988773>"),
 MM("M<y_416h*x_0dc1h*d_753h*p_145960881*l_1*p_70118400*l_2*p_171684480*l_1*t_1*l_1*p_1457280*l_2*p_32112522*l_1*t_1*l_2*p_1858560*l_2*p_464928*l_1*p_3785280*t_1*l_2*p_2956800*l_1*p_42668368*t_1*l_2*p_1985280*l_1*p_85835157*l_2*p_951360*t_2*l_2*p_1985280*l_1*p_42707946*l_2*p_10665600>"),
 MM("M<y_13bh*x_1f22h*d_67ah*p_241761356*l_2*p_2344320*l_2*p_11598944*t_2*l_2*p_2787840*l_2*p_32549296*l_1*t_1*l_1*p_1394880*l_2*p_21312*l_1*p_97920*t_2*l_2*p_2830080*l_2*p_64043906*t_2*l_2*p_2386560*l_2*p_106699777*t_1*l_2*p_1457280*l_1*p_132852*t_1*l_2*p_2830080*l_2*p_42663560>"),
 MM("M<y_1e8h*x_867h*d_0f4ah*p_201505057*l_1*p_2999040*l_1*p_1906064*l_2*t_1*l_2*p_1858560*l_1*p_10665888*l_1*p_1083840*t_1*l_1*p_1499520*l_2*p_32999643*l_2*t_1*l_2*p_2386560*l_2*p_96033260*t_1*l_1*p_1394880*l_1*p_4176*l_2*p_4353600*t_1*l_2*p_1943040*l_2*p_64020835*t_1*l_2*p_2830080*l_2*p_43604473>"),
 MM("M<y_6b4h*x_1b19h*d_323h*p_192657058*l_1*p_1457280*l_2*p_32551155*l_1*t_2*l_2*p_1457280*l_1*p_11661639*l_2*t_1*l_1*p_3840*l_2*p_464928*l_2*p_1948800*t_1*l_2*p_5160000*t_2*l_2*p_1985280*l_1*p_42671266*t_1*l_1*p_21120*l_1*p_43697952*t_2*l_2*p_1943040*l_2*p_42831985>"),
 MM("M<y_6dah*x_9d9h*d_0c04h*p_185412182*l_1*p_2640000*l_1*p_33013873*l_1*t_1*l_2*p_1943040*l_2*p_22769632*l_2*t_1*l_1*p_2999040*l_1*p_1419111*l_1*t_2*l_2*p_3020160*l_2*t_1*l_2*p_2830080*l_2*p_42836757*t_2*l_2*p_1943040*l_2*p_43161937*t_1*l_2*p_2956800*l_1*p_43196548*t_1*l_2*p_2344320*l_2*p_1937>"),
 MM("M<y_1cbh*x_0c7ch*d_7ech*p_161862937*l_2*p_1993920*l_1*t_2*l_1*p_1499520*l_2*p_469607*l_2*t_1*l_1*p_1457280*l_2*p_32126917*l_2*t_1*l_2*p_2344320*l_2*p_576391*t_2*l_2*p_2386560*l_2*p_64043946*t_1*l_2*p_1943040*l_2*p_64083394*t_1*l_2*p_1943040*l_2*p_170653290>"),
 MM("M<y_300h*x_0ae5h*d_90eh*p_173923830*l_1*p_2999040*l_1*p_1901092*l_1*t_1*l_2*p_1985280*l_1*p_21356312*t_2*l_1*p_1499520*l_2*p_64006454*t_1*l_2*p_2830080*l_2*p_11172914*t_2*l_2*p_1985280*l_1*p_42664532*t_2*l_2*p_1943040*l_2*p_10884082*t_1*l_2*p_2787840*l_2*p_12501>"),
 MM("M<y_432h*x_17cbh*d_6e5h*p_13411729*l_1*p_2640000*l_1*p_11236336*t_1*l_2*p_1457280*l_1*p_12108843*l_1*t_1*l_2*p_1393920*l_1*p_21360*l_2*p_65280*t_1*l_2*p_1943040*l_2*p_21446770*t_1*l_2*p_2956800*l_1*p_42834820*t_1*l_2*p_2597760*l_1*p_53442840*t_2*l_2*p_2386560*l_2*p_127989696>"),
 MM("M<y_17eh*x_1225h*d_2f5h*p_194093027*l_2*p_2597760*l_1*p_12153200*l_1*t_1*l_2*p_1920*l_2*p_24288*l_2*p_2899200*t_2*l_2*p_2787840*l_2*p_1501889*l_2*t_2*l_1*p_69231360*l_1*p_180998496*t_1*l_1*p_1920*l_2*p_10667760*l_1*p_2494080*t_2*l_2*p_3345600*l_2*t_2*l_1*p_1499520*l_1*p_42797315>"),
 MM("M<y_511h*x_1fe3h*d_3edh*p_155484770*l_2*p_2830080*l_2*p_11599915*l_2*t_2*l_2*p_2386560*l_2*p_32065409*l_1*t_2*l_1*p_1499520*l_1*p_33000454*l_2*t_2*l_2*p_2597760*l_1*p_43172512*t_2*l_2*p_1985280*l_1*p_42677970*t_1*l_2*p_2597760*l_1*p_10704132*t_1*l_2*p_1900800*l_2*p_58717>"),
 MM("M<y_174h*x_0dc6h*d_541h*p_132000462*l_2*p_1985280*l_1*p_1904164*t_2*l_1*p_2027520*l_1*p_10676234*t_1*l_1*p_1457280*l_2*p_489745*t_1*l_2*p_2956800*l_1*p_21873114*t_1*l_2*p_2386560*l_2*p_53394651*t_1*l_2*p_2787840*l_2*p_966875*t_2*l_2*p_1887360*l_2>"),
 MM("M<y_45ch*x_12ah*d_8a0h*p_193057876*l_2*p_1858560*l_1*p_32461680*l_2*t_2*l_2*p_1900800*l_2*p_12577512*l_2*t_1*l_2*p_1393920*l_1*p_10665936*l_1*p_2492160*t_2*l_2*p_2597760*l_1*p_64015158*t_2*l_2*p_1457280*l_1*p_96464230*l_1*p_10688640*t_2*l_1*p_2640000*l_1*p_10729406>"),
 MM("M<y_5b3h*x_0d7ch*d_67h*p_144713992*l_2*p_1457280*l_1*p_10674450*t_2*l_2*p_2787840*l_2*p_12059876*l_2*t_2*l_2*p_3840*l_1*p_4176*l_1*p_6065280*t_2*l_2*p_2787840*l_2*p_500259*t_1*l_2*p_2597760*l_1*p_96039893*l_2*p_1920*t_2*l_2*p_2597760*l_1*p_95252*l_2>"),
 MM("M<y_152h*x_191bh*d_0ef5h*p_70582253*l_1*p_2999040*l_1*p_22372306*t_1*l_1*p_464640*l_1*p_32504976*l_2*t_1*l_2*p_1457280*l_1*p_12594725*l_2*t_1*l_2*p_2597760*l_1*p_42677960*t_1*l_2*p_2386560*l_2*p_63998743*t_2*l_2*p_24000*l_2*p_64015104*t_2*l_1*p_1499520*l_2*p_21339059*t_1*l_2*p_2597760*l_1*p_53804500>"),
 MM("M<y_585h*x_1da5h*d_0b8ch*p_77678700*l_1*p_60804480*l_1*p_241760688*l_1*t_2*l_1*p_6673920*l_2*p_1795200*l_2*t_2*l_1*p_1457280*l_2*p_32576201*l_1*t_2*l_2*p_2956800*l_1*p_42677017*t_1*l_2*p_2830080*l_2*p_21428544*t_1*l_2*p_2597760*l_1*p_43153347*t_2*l_2*p_1943040*l_2*p_64001589*t_2*l_2*p_2830080*l_2*p_85328069>"),
 MM("M<y_1e5h*x_1d11h*d_9a2h*p_11573288*l_2*p_69674880*l_2*p_172571520*l_1*t_2*l_1*p_45281280*l_1*p_11552640*l_2*t_2*l_1*p_2640000*l_1*p_11691312*l_1*t_1*l_2*p_1943040*l_2*p_85326181*t_1*l_2*p_1943040*l_2*p_85819766*t_2*l_1*p_3840*l_2*p_10665840*l_1*p_10392960*t_1*l_1*p_3888000*l_1*t_1*l_2*p_1943040*l_2*p_85836130>"),
 MM("M<y_10bh*x_1270h*d_2c3h*p_234855125*l_2*p_2830080*l_2*p_63999698*t_2*l_2*p_2956800*l_1*p_22371331*l_2*t_2*l_2*p_2956800*l_1*p_475282*l_1*t_2*l_2*p_1943040*l_2*p_64024704*t_1*l_2*p_2787840*l_2*p_14427*t_1*l_2*p_1943040*l_2*p_85840917*t_2*l_1*p_49272960*l_2*p_161927328>"),
 MM("M<y_414h*x_263h*d_0e07h*p_31300264*l_1*p_2027520*l_1*p_11625824*l_1*t_1*l_2*p_2956800*l_1*p_1868386*l_2*t_1*l_2*p_1943040*l_2*p_1912821*l_1*t_2*l_2*p_1900800*l_2*p_6753*t_1*l_2*p_2597760*l_1*p_21344849*t_2*l_1*p_1499520*l_1*p_42670276*t_1*l_2*p_1985280*l_1*p_21363048*t_2*l_1*p_1457280*l_2*p_64790>"),
 MM("M<y_13dh*x_985h*d_1aah*p_198254363*l_2*p_1985280*l_1*p_1306*l_1*t_1*l_2*p_1415040*l_2*p_4176*l_1*p_178560*t_1*l_2*p_2386560*l_2*p_32548356*l_2*t_2*l_1*p_1499520*l_1*p_42711747*t_1*l_2*p_1457280*l_1*p_19201*t_1*l_2*p_1943040*l_2*p_21364949*t_1*l_2*p_1900800*l_2*p_12483>"),
 MM("M<y_157h*x_1c83h*d_5cbh*p_182349983*l_1*p_80319360*l_1*p_241317120*l_1*t_2*l_2*p_1943040*l_2*p_53378263*t_2*l_1*p_1499520*l_1*p_10698433*t_1*l_2*p_1943040*l_2*p_21344848*t_1*l_1*p_1499520*l_2*p_43177363*t_2*l_1*p_2640000*l_1*p_56803*t_2*l_1*p_2640000*l_1*p_15378>"),
 MM("M<y_593h*x_87ch*d_0a5ch*p_203745286*l_2*p_1985280*l_1*p_520550*t_1*l_2*p_1943040*l_2*p_12592817*l_1*t_2*l_2*p_2880*l_2*p_465792*l_2*p_3403200*t_1*l_1*p_2027520*l_1*p_932258*t_2*l_1*p_1499520*l_1*p_53455310*t_2*l_2*p_464640*l_1*p_53455296*t_2*l_2*p_2386560*l_2*p_53376384>"),
 MM("M<y_11bh*x_1646h*d_640h*p_181462845*l_2*p_2344320*l_2*p_11688473*t_2*l_2*p_1985280*l_1*p_11682595*l_1*t_1*l_1*p_1394880*l_2*p_24192*l_1*p_1587840*t_1*l_1*p_1457280*l_2*p_71220*t_1*l_1*p_1415040*l_2*p_23376*l_1*p_722880*t_2*l_2*p_23040*l_2*p_43170816*t_1*l_2*p_2386560*l_2*p_64085323>"),
 MM("M<y_0d7h*x_14dah*d_202h*p_189002550*l_1*p_70561920*l_2*p_12439680*l_2*t_2*l_1*p_1499520*l_2*p_22820779*l_2*t_2*l_2*p_2880*l_2*p_2160*l_2*p_10460160*t_1*l_2*p_2344320*l_2*p_75051*t_2*l_1*p_1393920*l_2*p_464832*l_1*p_1589760*t_1*l_1*p_8004480*l_1*t_1*l_2*p_6043200*l_1>"),
 MM("M<y_0ech*x_17f8h*d_7aeh*p_56919296*l_2*p_2956800*l_1*p_53440939*t_1*l_2*p_1943040*l_2*p_23233411*l_1*t_2*l_1*p_1499520*l_2*p_12171496*l_1*t_2*l_1*p_2999040*l_1*p_2800777*t_2*l_2*p_1900800*l_2*p_85414656*l_2*t_1*l_1*p_1394880*l_1*p_23232*l_2*p_6107520>"),
 MM("M<y_61h*x_32bh*d_2fch*p_58587268*l_2*p_1900800*l_2*p_521507*t_2*l_2*p_1457280*l_1*p_5912*l_2*t_2*l_2*p_1457280*l_1*p_2800897*l_1*t_1*l_2*p_2386560*l_2*p_64016096*t_1*l_2*p_2830080*l_2*p_53904595*t_1*l_2*p_1457280*l_1*p_12485*t_1*l_2*p_2830080*l_2*p_42792514>")]

Check that these involutions do invert $g_5$, and that they commute with $c_2$, so that the resulting subgroups $\mathrm{PSL}_2(16)$ are not maximal in $\mathrm{M}$. (One can further check that each involution extends $A_B$ to $\mathrm{PSL}_2(16)$ as described in Remark 5.1. We omit this check here.)

In [60]:
all([g5_B**x == g5_B**-1 and Comm(c2_B,x) == MM("M<1>") for x in involutions_inverting_g5_B])

True

## Code accompanying Section 6

All elements from the "type G" case of Listing 8 (and define $g_5=g_2g_3$).

Drop the subscript "G" from the variable names now.

In [61]:
g2 = MM("M<y_4f6h*x_1f98h*d_0b7h*p_67615847*l_1*p_2999040*l_1*p_86264262*l_2*p_11172480>")
g3 = MM("M<y_4e1h*x_19cbh*d_9c8h*p_19643307*l_1*p_2999040*l_1*p_64003504*l_2*p_1478400>")
g5 = g2*g3
c2 = MM("M<x_1000h>")
c5 = MM("M<y_548h*x_34ah*d_0a9ch*p_243281095*l_1*p_1457280*l_2*p_43255315*t_2*l_1*p_3840*l_2*p_465936*l_2*p_1101120*t_1*l_2*p_2787840*l_2*p_32009429*l_1*t_2*l_2*p_2956800*l_1*p_64018007*t_1*l_2*p_2880*l_2*p_3120*l_2*p_2579520*t_2*l_2*p_2830080*l_2*p_42706069*t_1*l_2*p_2787840*l_2*p_148289>")
hu = MM("M<y_51h*x_319h*d_0d15h*p_65451314*l_2*p_2344320*l_2*p_23241234*l_1*t_1*l_1*p_2027520*l_1*p_33397753*l_1*t_1*l_2*p_1393920*l_1*p_10666752*l_1*p_3847680*t_2*l_2*p_1943040*l_2*p_21374615*t_1*l_2*p_2880*l_2*p_10666896*l_2*p_2959680*t_1*l_1*p_1499520*l_1*p_85838981*t_1*l_2*p_506880>")
hv = MM("M<y_152h*x_33ah*d_0f3h*p_183194050*l_2*p_71005440*l_2*p_85198272*t_1*l_2*p_2386560*l_2*p_2355444*l_1*t_1*l_2*p_1920*l_2*p_467712*l_2*p_10371840*t_2*l_2*p_2597760*l_1*p_96018816*t_2*l_2*p_1985280*l_1*p_21419925*t_1*l_2*p_1900800*l_2*p_135669*t_2*l_1*p_1499520*l_1*p_64088183>")
j2 = MM("M<y_538h*x_170fh*d_0a21h*p_129570785*l_1*p_1499520*l_2*p_12610123*t_2*l_1*p_2640000*l_1*p_1867433*l_1*t_2*l_2*p_1900800*l_2*p_2798997*l_2*t_2*l_1*p_1499520*l_2*p_127989729*t_2*l_2*p_2830080*l_2*p_21352704*t_2*l_2*p_1985280*l_1*p_127995457*t_1*l_2*p_2597760*l_1*p_64002648*t_1*l_2*p_1943040*l_2*p_96485380>")
a12 = MM("M<y_578h*x_309h*d_1f8h*p_113794596*l_1*p_2027520*l_1*p_10863009*t_1*l_1*p_2999040*l_1*p_1871269*l_2*t_1*l_1*p_1457280*l_2*p_12556433*l_1*t_1*l_1*p_2999040*l_1*p_6743*t_1*l_1*p_1499520*l_1*p_127990661*t_1*l_1*p_1858560*l_1*p_21408*l_2*p_240000*t_2*l_1*p_4797120*l_1*t_2*l_2*p_2830080*l_2*p_106700769>")

We have already checked (above) that $c_2$ centralises $\langle g_2,g_3 \rangle$, and that $g_2 \in 2\text{B}$, $g_3 \in 3\text{C}$, and $g_5 \in 5\text{B}$. Check also that $c_2$ inverts $c_5$.

In [62]:
c5**c2 == c5**-1

True

Construct the 'good' element $u_5 = g_5c_5$ of order $5$, and check that $j_2 \in 2\text{B}$ and inverts $u_5$.

In [63]:
u5 = g5*c5
u5.order() == 5 and j2.conjugate_involution()[0] == 2 and u5**j2 == u5**-1

True

Check that $j_2$ and $g_3$ satisfy the given presentation for $\mathrm{PSU}_3(4)$.

In [64]:
j2**2 == g3**3 == (j2*g3**-1*j2*g3)**5 == (j2*g3)**15 == ((j2*g3)**3*(j2*g3**-1)**3)**3 == (j2*g3**-1*(j2*g3)**5)**4 == MM("M<1>")

True

Check that $c_5$ and $j_2$ do not commute, so that $S = \langle g_2,g_3,c_5,j_2 \rangle \cong \mathrm{PSU}_3(4)$ has trivial centraliser in $\mathrm{M}$.

In [65]:
Comm(c5,j2) == MM("M<1>")

False

Construct $S \cong \mathrm{PSU}_3(4)$ and confirm that it has order $62400$.

***Warning: this takes about an hour!***

In [66]:
S = group_generated_by([g2,g3,c5,j2], 100000)

In [67]:
len(S)

62400

To speed up certain calculations, also create a version of S in "tuples" format.

In [68]:
S_tuples = [x.as_tuples() for x in S]

### Proof of Proposition 6.1

Check that $a_{12}$ has order $12$ and does not lie in $S$.

***Here and for other checks below we use the "tuples" version of $S$ for efficiency; direct checks are also possible but take longer.***

In [69]:
a12.order() == 12 and not(a12.as_tuples() in S_tuples)

True

Check that $a_{12}$ normalises $S$ by showing that it conjugates each generator into $S$.

In [70]:
all([(x**a12).as_tuples() in S_tuples for x in [g2,g3,c5,j2]]) 

True

### Proof of Proposition 6.2

Per the proof, we only need to check certain conjugacy classes computationally. Desired $\chi_\mathrm{M}$-values are indicated in the proof.

(Note also that we do not need to enumerate $U = \langle S,a_{12} \rangle \cong \mathrm{PSL}_3(4){:}4$.)

Check that $a_{12}^6 \in 2\text{B}$ and $a_{12}^2 \in 6\text{F}$.

In [71]:
(a12**6).conjugate_involution()[0] == 2 and ((a12**2)**((a12**6).conjugate_involution()[1])).chi_G_x0()[0] == -1

True

Check that $g_4 = j_2g_3^{-1}(j_2g_3)^5 \in 4\text{C}$.

In [72]:
g4 = j2*g3**-1*(j2*g3)**5
g4.order() == 4 and (g4**((g4**2).conjugate_involution()[1])).chi_G_x0()[0] == 19

True

Check that $a_{12}^3 \in 4\text{D}$.

In [73]:
((a12**3)**((a12**6).conjugate_involution()[1])).chi_G_x0()[0] == -13

True

Check that $g_{10} = c_5^3g_2 \in 10\text{E}$.

In [74]:
g10 = c5**3*g2
g10.order() == 10 and (g10**((g10**5).conjugate_involution()[1])).in_G_x0() and (g10**((g10**5).conjugate_involution()[1])).chi_G_x0()[0] == 0

True

Check that $h_{10} = a_{12}^2c_5 \in 10\text{E}$.

In [75]:
h10 = a12**2*c5
h10.order() == 10 and (h10**((h10**5).conjugate_involution()[1])).in_G_x0() and (h10**((h10**5).conjugate_involution()[1])).chi_G_x0()[0] == 0

True

Check that the involution $h_{10}^5$ is centralised by an element of order $3$, namely the element $g_3^{j_2}[h_{10}^5,g_3^{j_2}]^2$. (Per the proof, this shows that $g_{10}$ and $h_{10}$ are not conjugate in $U$.)

In [76]:
k3 = g3**j2*Comm(h10**5, g3**j2)**2
k3.order() == 3 and Comm(k3,h10**5) == MM("M<1>")

True

Check that $h_{10}^2 \in 5\text{B}$.

In [77]:
((h10**2)**((h10**5).conjugate_involution()[1])).chi_G_x0()[0] == 8

True

Check that $g_{16} = a_{12}j_2 \in 16\text{C}$.

In [78]:
g16 = a12*j2
g16.order() == 16 and (g16**((g16**8).conjugate_involution()[1])).chi_G_x0()[0] == 7

True

Check that $h_{16} = c_5^3a_{12}j_2 \in 16\text{B}$.

In [79]:
h16 = c5**3*a12*j2
h16.order() == 16 and (h16**((h16**8).conjugate_involution()[1])).chi_G_x0()[0] == -1

True

Check that $g_{16}^2 \in 8\text{E}$.

In [80]:
((g16**2)**((g16**8).conjugate_involution()[1])).chi_G_x0()[0] == 3

True

Check that $g_{13} = j_2g_2g_3^{-1}$ has order $13$.

In [81]:
g13 = j2*g2*g3**-1
g13.order()

13

In [82]:
g13

M<y_5c6h*x_0d81h*d_18bh*p_15333659*l_1*p_1499520*l_2*p_21331893*t_2*l_2*p_2597760*l_1*p_32947494*t_2*l_2*p_2597760*l_1*p_42712609*t_2*l_2*p_2386560*l_2*p_21426658*t_1*l_1*p_1394880*l_1*p_10665888*l_1*p_1672320*t_2*l_2*p_2956800*l_1*p_42712633*t_1*l_1*p_1499520*l_1*p_42793459>

The following function is adapted from code kindly provided by Gerald Hoehn, which he has now made publicly available: 

http://www.monstrous-moonshine.de/~gerald/monster/

Per the proof of Proposition 6.2, it shows that $g_{13} \in 13\text{B}$ by calculating the trace of $g_{13}$ in its action on the reducible $\mathbb{F}_3\mathbf{M}$-module of dimension $196884$.

In [83]:
# define the reducible M-module V of dimension 196884 over the field of order 3
V = MMV(3)

# auxiliary function
def partial_trace_mod_3(start, end, g):
    return sum([(V([('E', i)]) * g)['E'][i] % 3 for i in range(start, end)])

# function to calculate the trace of g acting on V
def trace_mod_3(g, n_processes=1):
    chunk_size = 196884 // n_processes
    start_indices = [i * chunk_size for i in range(n_processes)]
    end_indices = [(start + chunk_size) if i < n_processes - 1 else 196884 for i, start in enumerate(start_indices)]
    with mp.Pool(n_processes) as pool:
        results = pool.starmap(partial_trace_mod_3, [(start, end, g) for start, end in zip(start_indices, end_indices)])
    return sum(results) %3

Check. Per the proof, the output should be $-1$ mod $3$.

***Warning: this can take a while! It also does not seem to run properly in Jupyter Notebook due to some issue with the multiprocessing package, so it is recommended to run it in a terminal. Given that it also does not affect any subsequent calculations, we have commented it out here.***

In [84]:
#trace_mod_3(g13, 100)

## Code accompanying Section 7

The elements from Listing 9.

In [85]:
e1 = MM("M<x_1920h*d_4c8h>")
e2 = MM("M<x_1d90h*d_5b0h>")
e3 = MM("M<x_0ec8h*d_3c5h>")
g7 = MM("M<y_5d3h*x_0a6dh*d_8d4h*p_111142481*l_1*p_2999040*l_1*p_43234193>")

### Proof of Lemma 7.1

Check the $H_4$ case, i.e. that $\langle z,y \rangle \cong 2^2$ is $2\text{B}$-pure and singular, and that the given element of order $7$ (called $h_7$ here) is of class $7\text{A}$ (via $\chi_\mathbf{M}$-value $50$).

In [86]:
z = MM("M<x_1000h>")
y = MM("M<x_800h>")
h7 = MM("M<y_3d6h*x_125dh*d_259h*p_81455268>")
y.in_Q_x0() and (z*y).in_Q_x0() and Comm(h7,z) == Comm(h7,y) == MM("M<1>") and h7.in_G_x0() and h7.order() == 7 and all([(h7**i).chi_G_x0()[0] == 50 for i in [1,2,3,4,5,6]])

True

### Proof of Proposition 7.2

Check that $e_i \in \mathbf{Q} \cap 2\text{B}$ for $i \in \{1,2,3\}$ and that $g_7 \in \mathbf{G} \cap 7\text{B}$. (Note that $7\text{B}$-elements have $\chi_\mathbf{M}$-value $1$.)

In [87]:
all([e.in_G_x0() and e.order() == 2 and e.conjugate_involution()[0] == 2 for e in [e1,e2,e3]]) and g7.in_G_x0() and g7.chi_G_x0()[0] == 1 

True

Calculate the image $m_7$ of $g_7$ under the projection $\pi: \mathbf{G} \rightarrow \mathrm{Co}_1 < \mathrm{GL}_{24}(2)$.

In [88]:
m7 = elt_to_24_mat(g7)

In [89]:
m7

[[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1],
 [0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0],
 [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1],
 [0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
 [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 0,

The Magma file

https://github.com/melissa-maths/M/blob/main/L28.m

calculates the orbits of $N_{\mathrm{Co}_1}(m_7)$ on $1$-dimensional subspaces of $2^{24}$. 

The following is a list of the (230) orbit representatives. (One needs to copy the output generated by the Magma file and insert commas into the vectors.)

In [90]:
orbit_rep_vectors = [
    (1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,0,1,1,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0),
    (1,0,1,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0),
    (1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,1,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0),
    (1,1,1,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0),
    (1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0)
]

The following code converts the (vector) orbit representatives to (mmgroup) elements of $\mathbf{Q} < \mathbf{M}$.

In [91]:
def vector_to_integer(v):
    return sum([2**(23-i)*v[i] for i in range(24)])
    
def integer_to_element(x):
    return MM(XLeech2(x))

orbit_rep_mmgroup = [integer_to_element(vector_to_integer(v)) for v in orbit_rep_vectors]

In [92]:
all([x.in_Q_x0() for x in orbit_rep_mmgroup])

True

The following code checks the preimage $x \in \mathbf{Q} < \mathbf{M}$ of each orbit representative to determine whether either element of the coset $xZ(\mathbf{Q})$ extends $\langle g_7 \rangle$ to a group of order $|2^3{:}7|=56$.

In [93]:
good_2B = []
for x in orbit_rep_mmgroup:
    if x.order() == 2 and x.conjugate_involution()[0] == 2 and group_generated_by([g7,x], 100, order_only=True) == 56:
        good_2B.append(x)
    x = x*MM("M<x_1000h>")
    if x.order() == 2 and x.conjugate_involution()[0] == 2 and group_generated_by([g7,x], 100, order_only=True) == 56:
        good_2B.append(x)

Group is larger than imposed limit -- abort in time  0.0194
Group is larger than imposed limit -- abort in time  0.0097
Group is larger than imposed limit -- abort in time  0.0089
Group is larger than imposed limit -- abort in time  0.0084
Group is larger than imposed limit -- abort in time  0.0083
Group is larger than imposed limit -- abort in time  0.0087
Group is larger than imposed limit -- abort in time  0.0087
Group is larger than imposed limit -- abort in time  0.0086
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0074
Group is larger than imposed limit -- abort in time  0.0075
Group is larger than imposed limit -- abort in time  0.0073
Group is larger than imposed limit -- abort in time  0.0074
Group is larger than imposed limit -- abort in time  0.0074
Group is larger than imposed limit -- abort in time  0.0075
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- ab

Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0098
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0098
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0102
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abor

Group is larger than imposed limit -- abort in time  0.0106
Group is larger than imposed limit -- abort in time  0.0105
Group is larger than imposed limit -- abort in time  0.0101
Group is larger than imposed limit -- abort in time  0.0103
Group is larger than imposed limit -- abort in time  0.0104
Group is larger than imposed limit -- abort in time  0.0101
Group is larger than imposed limit -- abort in time  0.0084
Group is larger than imposed limit -- abort in time  0.0082
Group is larger than imposed limit -- abort in time  0.0083
Group is larger than imposed limit -- abort in time  0.0104
Group is larger than imposed limit -- abort in time  0.0081
Group is larger than imposed limit -- abort in time  0.0085
Group is larger than imposed limit -- abort in time  0.0082
Group is larger than imposed limit -- abort in time  0.0083
Group is larger than imposed limit -- abort in time  0.008
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- abo

Group is larger than imposed limit -- abort in time  0.0104
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0098
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0102
Group is larger than imposed limit -- abort in time  0.0075
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abort in time  0.0077
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abort in time  0.008
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0097
Group is larger than imposed limit -- abort in time  0.0075
Group is larger than imposed limit -- abort i

Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0093
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0089
Group is larger than imposed limit -- abort in time  0.0089
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abort in time  0.0082
Group is larger than imposed limit -- abort in time  0.0083
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abort in t

Group is larger than imposed limit -- abort in time  0.0101
Group is larger than imposed limit -- abort in time  0.0104
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0102
Group is larger than imposed limit -- abort in time  0.0106
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0098
Group is larger than imposed limit -- abort in time  0.0094
Group is larger than imposed limit -- abort in time  0.0083
Group is larger than imposed limit -- abort in time  0.0087
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abort in time  0.0088
Group is larger than imposed limit -- abort in time  0.008
Group is larger than imposed limit -- abort in time  0.0087
Group is larger than imposed limit -- abort in time  0.009
Group is larger than imposed limit -- abort in time  0.0091
Group is larger than imposed limit -- abort 

Group is larger than imposed limit -- abort in time  0.0108
Group is larger than imposed limit -- abort in time  0.0104
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0104
Group is larger than imposed limit -- abort in time  0.0098
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.0106
Group is larger than imposed limit -- abort in time  0.0086
Group is larger than imposed limit -- abort in time  0.0086
Group is larger than imposed limit -- abort in time  0.0091
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0093
Group is larger than imposed limit -- abort in time  0.0084
Group is larger than imposed limit -- abort in time  0.0087
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0086
Group is larger than imposed limit -- abor

Group is larger than imposed limit -- abort in time  0.0101
Group is larger than imposed limit -- abort in time  0.0104
Group is larger than imposed limit -- abort in time  0.0103
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0099
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0101
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0101
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.01
Group is larger than imposed limit -- abort in time  0.0106
Group is larger than imposed limit -- abort in time  0.0103
Group is larger than imposed limit -- abort in time  0.008
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abort in time  

Group is larger than imposed limit -- abort in time  0.0103
Group is larger than imposed limit -- abort in time  0.0094
Group is larger than imposed limit -- abort in time  0.0102
Group is larger than imposed limit -- abort in time  0.008
Group is larger than imposed limit -- abort in time  0.0081
Group is larger than imposed limit -- abort in time  0.0081
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.009
Group is larger than imposed limit -- abort in time  0.0079
Group is larger than imposed limit -- abort in time  0.0078
Group is larger than imposed limit -- abort in time  0.0077
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- abort in time  0.0076
Group is larger than imposed limit -- abort in time  0.0077
Group is larger than imposed limit -- abor

Group is larger than imposed limit -- abort in time  0.0128


Confirm that the only elements that pass this test are $e_1$, $e_2$, and $e_3$.

In [94]:
len(good_2B) == 3 and all(x in good_2B for x in [e1,e2,e3])

True

Confirm that the orbit of each $e_i$ under conjugation by $g_7$ has length $7$, so that each $B_i = \langle e_i,g_7 \rangle$ has the form $2^3{:}7$.

In [95]:
E1 = [e1**(g7**i) for i in range(g7.order())]
E2 = [e2**(g7**i) for i in range(g7.order())]
E3 = [e3**(g7**i) for i in range(g7.order())]
len([x.as_tuples() for x in E1]) == len([x.as_tuples() for x in E2]) == len([x.as_tuples() for x in E3]) == 7

True

Confirm that the normal subgroup $2^3$ of each $B_i$ is $2\text{B}$-pure.

In [96]:
all(x.conjugate_involution()[0] == 2 for x in E1+E2+E3)

True

### Remark 7.3

Check that subgroups $2^2$ of distinct $B_i$ are conjugate in $\mathbf{M}$. Per Remark 7.3, it suffices to take two involutions in each normal $2^3 < B_i$, conjugate one to $z \in Z(\mathrm{G})$, and calculate the $\chi_{299}$-value of the corresponding conjugate of the other. The character values should be the same (namely $43$) for each $B_i$.

In [97]:
pairs = [E1[:2], E2[:2], E3[:2]]
all([(p[1]**(p[0].conjugate_involution()[1])).chi_G_x0()[1] == 43 for p in pairs])

True

### Remark 7.4

Finally, here are generators for the `large' subgroup $Y$ of $N_\mathbf{M}(g_7)$ (with check). As in previous cases, the involutions "$j_2$" inverting $g_7$ can be be recovered via random search in $\langle Y \rangle$. (We do not provide these directly due to space constraints.)

In [98]:
normaliser_of_g7 = [MM("M<y_96h*x_16d3h*d_10ch*p_201864567*l_2*p_2344320*l_2*p_33419049*t_2*l_2*p_2830080*l_2*p_31998112*l_2*t_2*l_1*p_2999040*l_1*p_21868145*l_2*t_2*l_1*p_1499520*l_2*p_96456601*t_2*l_2*p_2386560*l_2*p_42832937*t_2*l_2*p_1943040*l_2*p_43237027*t_2*l_2*p_1943040*l_2*p_64015144>"),
 MM("M<y_15ah*x_0d3h*d_9cah*p_133162528*l_2*p_2386560*l_2*p_22816918*l_2*t_1*l_1*p_2999040*l_1*p_22355058*l_1*t_2*l_1*p_2640000*l_1*p_22759184*l_1*t_1*l_1*p_1457280*l_2*p_12547*t_2*l_1*p_1499520*l_1*p_64081481*t_1*l_1*p_1499520*l_2*p_43154242*t_1*l_2*p_1943040*l_2*p_127992599>"),
 MM("M<y_37ch*x_0c2fh*d_0e63h*p_10883322*l_2*p_1985280*l_1*p_32512858*l_1>"),
 MM("M<y_39bh*x_0b51h*d_0ef7h*p_6649870*l_1*p_1499520*l_1*p_1464466*l_1>"),
 MM("M<y_5d3h*x_0a6dh*d_8d4h*p_111142481*l_1*p_2999040*l_1*p_43234193>"),
 MM("M<y_592h*x_0b0h*d_6e3h*p_149342936*l_1*p_1457280*l_2*p_42711690*l_1*p_22767360>")]

In [99]:
all([is_normalised(g7,h) for h in normaliser_of_g7])

True